In [73]:
import glob
import os
import pandas as pd
import numpy as np
import slugify
import re

from google_trans_new import google_translator
translator = google_translator(proxies={'https': '102.134.26.41:3128',
                                       'https': '167.86.78.204:3128',
                                       'https': '143.198.192.193:80',
                                       'https': '186.10.80.122:53281',
                                       'https': '172.107.159.194:443'})  

## Get functions ready

In [2]:
def read_sheet(dataset, tablename):
    '''reads sheet in an excel dataset and removes nan rows'''
    
    df = dataset.parse(tablename)
    df = df.dropna(axis=0, how='all')
    df.reset_index(inplace=True, drop=True)
    return df

def clean_headers(df, header_row):
    '''combines all header rows into a single one'''
    
    for col in df.keys():
        df.at[0, col] = (' ').join(x for x in list(df[col][0:header_row]) if str(x) != 'nan')
    df.columns = df.iloc[0]
    df.drop(df.index[0:header_row], inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df

def year_region(df, yr_col = 'Year Region', std_year = 2019, std_region = 'China'):
    '''separates Year Region column into two separate columns;
    assumes that values for years are for the whole country, and that values for regions are for 2019'''
    
    df['Region'] = ''
    for row in df.index:
        try:
            float(df[yr_col][row])
            df.at[row, 'Region'] = std_region
        except:
            df.at[row, 'Region'] = df.iloc[row][yr_col]
            df.at[row, yr_col] = std_year
    df.rename(columns={yr_col: 'Year'}, inplace=True)
    return df

def norm_df(df, cat_dic, unit_dic, filename, tablename):
    '''normalizes dataframe'''
    
    table_df = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])
    for col in df.keys():
        if col != 'Region' and col != 'Year':
            temp_df = df[['Region', 'Year', col]].copy()


            try:
                ind = col
                units = unit_dic[col]
            except:
                ind = col.split(' (')[0]
                units = col.split('(')[1].split(')')[0]


            if col in cat_dic.keys():
                if isinstance(cat_dic[col], list):
                    temp_df['indicator'] = cat_dic[col][0]
                    for item in range(1, len(cat_dic[col])):
                        temp_df[f'category_{item}'] = cat_dic[col][item]
                    if len(cat_dic[col]) < 4:
                        for item in range(len(cat_dic[col]), 4):
                            temp_df[f'category_{item}'] = None
                else:
                    temp_df['indicator'] = cat_dic[col]
                    temp_df['category_1'] = ind
                    for item in range(2, 4):
                        temp_df[f'category_{item}'] = None
            else:
                temp_df['indicator'] = ind
                for item in range(1, 4):
                        temp_df[f'category_{item}'] = None

            temp_df['units'] = units
            temp_df.rename(columns={col:'value'}, inplace=True)
            temp_df['file_name'] = filename#.split('_')[0]
            temp_df['table_name'] = tablename
            temp_df['region_type'] =  'Province'
            temp_df.loc[temp_df.Region == 'China', 'region_type'] = "Country"
            temp_df = temp_df[['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
                       'region_type', 'Year', 'value', 'units']]
            table_df = pd.concat([table_df, temp_df])
            table_df.reset_index(inplace=True, drop=True)
            
    for col in ['Year', 'value']:
        table_df[col] = pd.to_numeric(table_df[col])
    
    return table_df

def check_norm(df, info = True):
    """Extracts table summary from normalised dataset and all existing indicator-categories-unit combinations"""
    
    if info:
        print(df.info())
    cols = ['indicator', 'category_1']
    for cat in range(2,4):
        if len(list(df[f"category_{cat}"].unique())) == 1 and str(list(df[f"category_{cat}"].unique())[0]) == 'None':
            pass
        else:
            cols.append(f"category_{cat}")
    cols.append('units')
    
    summary_df = df.groupby(cols, sort=False, as_index=False, dropna=False).size()#.reset_index().rename(columns={0:'count'})
    return summary_df

## Import Datasets

In [3]:
#list of all files
input_path = '../data/Socio-economic Data' #add here the path to the directory where the datasets are stored
os.chdir(f"{input_path}")
file_list = []
# use os.walk(path) on the main path to get all filess inside path
for filename in glob.glob('*.xlsx'):
    #Read and name all files
    name = os.path.split(filename)[1].split(".")[0]
    file_list.append(name)

In [4]:
file_list = sorted(file_list)

In [5]:
print('Sheets per file:')
for file in file_list:
    dataset = pd.ExcelFile(f'{file}.xlsx')
    print(file, len(dataset.sheet_names))

Sheets per file:
Agriculture_农业 6
Building_建筑 2
Employment_就业 6
Exchange Rate_汇率 2
GDP_国内生产总值 27
Imports and exports_进出口 12
Industry_工业 2
Per capita income and consumption expenditure_人均收支 8
Population_人口 8
Transportation_交通 24


In [6]:
#We're going to append all normalised datasets to this list to eventually combine them into a single dataframe
dataframe_list = {}

### Agriculture Dataset

In [7]:
filename = file_list[0]
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

['12-4  主要农业机械拥有量 (年底数)',
 '12-5  耕地灌溉面积和农用化肥施用量',
 '12-6  灌溉、水库和除涝治水情况',
 '12-4  Major Agricultural Machin',
 '12-5  Irrigated Area of Cultiva',
 '12-6  Irrigation, Reservoirs, F']

In [31]:
tablename = sheets[3]
agric = read_sheet(dataset, tablename)
agric = clean_headers(agric, 7)
agric = year_region(agric)

In [32]:
ind_encn = {'Total Power of Agricultural Machinery': '农业机械 总 动 力',
           'Large and Medium-sized Tractors Number': '大中型拖拉机 数  量',
           'Towing Farm Machinery': '配套农具',
           'Small Tractors Number': '小 型 拖 拉 机 数  量'}
unit_encn = {'10 000 kw': '万千瓦', '10 000 units': '万台'}

In [33]:
cat_dic = {'Large and Medium-sized Tractors Number (10 000 units)': 'Farm Machinery',
       'Towing Farm Machinery (10 000 units)': 'Farm Machinery',
       'Small Tractors Number (10 000 units)': 'Farm Machinery'}
unit_dic = {}
norm_agric = norm_df(agric, cat_dic, unit_dic, filename, tablename)

In [34]:
check_norm(norm_agric)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    208 non-null    object 
 1   table_name   208 non-null    object 
 2   indicator    208 non-null    object 
 3   category_1   156 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       208 non-null    object 
 7   region_type  208 non-null    object 
 8   Year         208 non-null    int64  
 9   value        208 non-null    float64
 10  units        208 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 18.0+ KB
None


,indicator,category_1,units,size
0,Total Power of Agricultural Machinery,NaN,10 000 kw,52
1,Farm Machinery,Large and Medium-sized Tractors Number,10 000 units,52
2,Farm Machinery,Towing Farm Machinery,10 000 units,52
3,Farm Machinery,Small Tractors Number,10 000 units,52


In [35]:
dataframe_list['agric'] = norm_agric

In [54]:
tablename = sheets[4]
cult = read_sheet(dataset, tablename)

In [55]:
cult = clean_headers(cult, 6)

In [56]:
cult_cn = read_sheet(dataset, sheets[1])
cult_cn = clean_headers(cult_cn, 3)

In [57]:
for col, col_cn in zip(cult.keys()[1:], cult_cn.keys()[1:]):
    if col.find('(')>-1:
        ind_encn[col.split(' (')[0]] = col_cn.split(' (')[0]
        unit_encn[col.split(' (')[1].split(')')[0]] = col_cn.split(' (')[1].split(')')[0]
    else:
        ind_encn[col] = col_cn

In [58]:
cult = year_region(cult)

In [59]:
cat_dic = {'Nitrogenous Fertilizer': 'Consumption of Chemical Fertilizers',
           'Phosphate Fertilizer': 'Consumption of Chemical Fertilizers',
           'Potash Fertilizer': 'Consumption of Chemical Fertilizers',
           'Compound Fertilizer': 'Consumption of Chemical Fertilizers'}
unit_dic = {'Nitrogenous Fertilizer': '10 000 tons',
           'Phosphate Fertilizer': '10 000 tons',
           'Potash Fertilizer': '10 000 tons',
           'Compound Fertilizer': '10 000 tons'}
norm_cult = norm_df(cult, cat_dic, unit_dic, filename, tablename)

In [60]:
norm_cult.loc[(norm_cult['indicator'] == 'Consumption of Chemical Fertilizers')&(norm_cult['category_1'].isnull()), 'category_1'] = 'Total'

In [61]:
check_norm(norm_cult)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    312 non-null    object 
 1   table_name   312 non-null    object 
 2   indicator    312 non-null    object 
 3   category_1   260 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       312 non-null    object 
 7   region_type  312 non-null    object 
 8   Year         312 non-null    int64  
 9   value        308 non-null    float64
 10  units        312 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 26.9+ KB
None


,indicator,category_1,units,size
0,Irrigated Area of Cultivated Land,NaN,1 000 hectares,52
1,Consumption of Chemical Fertilizers,Total,10 000 tons,52
2,Consumption of Chemical Fertilizers,Nitrogenous Fertilizer,10 000 tons,52
3,Consumption of Chemical Fertilizers,Phosphate Fertilizer,10 000 tons,52
4,Consumption of Chemical Fertilizers,Potash Fertilizer,10 000 tons,52
5,Consumption of Chemical Fertilizers,Compound Fertilizer,10 000 tons,52


In [62]:
dataframe_list['cult'] = norm_cult

In [65]:
tablename = sheets[5]
res = read_sheet(dataset, tablename)

In [66]:
res_cn = read_sheet(dataset, sheets[2])

In [67]:
res.head(10)

,"12-6 Irrigation, Reservoirs, Flood Prevention, Water and Soil Conservation",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Item,2000.0,2005.0,2010.0,2015.0,2018.0000,2019.0000
1,Number of Irrigated Areas over 10 000 Mu ...,5683.0,5860.0,5795.0,7773.0,7881.0000,7884.0000
2,33 000 Hectares and Over,101.0,117.0,131.0,176.0,175.0000,176.0000
3,20 000-33 000 Hectares,141.0,170.0,218.0,280.0,286.0000,284.0000
4,Effective Irrigated Area ...,2449.3,2641.9,2941.5,3230.2,3332.4085,3350.0549
5,33 000 Hectares and Over,788.3,1023.0,1091.8,1202.4,1239.8704,1260.8528
6,20 000-33 000 Hectares,344.0,408.0,474.0,566.3,540.0184,538.5677
7,Number of Reservoirs ...,83260.0,85108.0,87873.0,97988.0,98822.0000,98112.0000
8,Large Reservoir,420.0,470.0,552.0,707.0,736.0000,744.0000
9,Medium-sized Reservoir,2704.0,2934.0,3269.0,3844.0,3954.0000,3978.0000


In [ ]:
for col, col_cn in zip(res[res.keys()[0]][1:], cult_cn.keys()[1:]):
    if col.find('(')>-1:
        ind_encn[col.split(' (')[0]] = col_cn.split(' (')[0]
        unit_encn[col.split(' (')[1].split(')')[0]] = col_cn.split(' (')[1].split(')')[0]
    else:
        ind_encn[col] = col_cn

In [154]:
header = res.iloc[0] #grab the first row for the header
res = res[1:] #take the data less the header row
res.columns = header #set the header row as the df header
res.reset_index(inplace=True, drop=True)

In [155]:
res.rename(columns={'Item': 'Year'}, inplace=True)

#Fix rows with same names
for row in range(11, 14):
    res.at[row, 'Year'] = f"Cap {res.iloc[row]['Year']}"

In [156]:
res.drop(axis=0, index=[19, 20], inplace=True)

In [157]:
res

,Year,2000.0,2005.0,2010.0,2015.0,2018.0,2019.0
0,Number of Irrigated Areas over 10 000 Mu ...,5683.0,5860.0,5795.0,7773.0000,7881.000000,7884.000000
1,33 000 Hectares and Over,101.0,117.0,131.0,176.0000,175.000000,176.000000
2,20 000-33 000 Hectares,141.0,170.0,218.0,280.0000,286.000000,284.000000
3,Effective Irrigated Area ...,2449.3,2641.9,2941.5,3230.2000,3332.408500,3350.054900
4,33 000 Hectares and Over,788.3,1023.0,1091.8,1202.4000,1239.870400,1260.852800
5,20 000-33 000 Hectares,344.0,408.0,474.0,566.3000,540.018400,538.567700
6,Number of Reservoirs ...,83260.0,85108.0,87873.0,97988.0000,98822.000000,98112.000000
7,Large Reservoir,420.0,470.0,552.0,707.0000,736.000000,744.000000
8,Medium-sized Reservoir,2704.0,2934.0,3269.0,3844.0000,3954.000000,3978.000000
9,Small Reservoir,80136.0,81704.0,84052.0,93437.0000,94132.000000,93390.000000


In [22]:
#Transpose to have indicators as columns and years as rows
res.set_index('Year', inplace=True)
res = res.transpose()
res.reset_index(inplace=True)

In [23]:
res.rename(columns = {0: 'Year'}, inplace=True)
res['Year'] = res['Year'].astype(int).astype(str)
res.index.rename('index', inplace=True)

In [24]:
#Remove spaces in col names
ren_dict = {}
for item in res.keys():
    if item.find('    ') > -1:
        ren_dict[item]= item.replace('    ', '')
    if item.find('   ') > -1:
        ren_dict[item]= item.replace('   ', '')
res.rename(columns=ren_dict, inplace=True)
res.keys()

Index(['Year', 'Number of Irrigated Areas over 10 000 Mu  (set)',
       '33 000  Hectares and Over', '20 000-33 000  Hectares',
       'Effective Irrigated Area  (10 000 hectares)',
       '33 000 Hectares and Over', '20 000-33 000 Hectares',
       'Number of Reservoirs  (unit)', ' Large Reservoir',
       ' Medium-sized Reservoir', ' Small Reservoir',
       'Capacity of Reservoirs (100 million cu.m)', 'Cap  Large Reservoir',
       'Cap  Medium-sized Reservoir', 'Cap  Small Reservoir',
       'Water-saving Irrigated Area  (10 000 hectares)',
       'Area with Flood Prevention Measures(10 000 hectares)',
       'Area with Soil Erosion under Control (10 000 hectares)',
       'Total Length of Dikes (10 000 km)',
       'Area of Land Protected by Dikes(10 000 hectares)'],
      dtype='object', name='Year')

In [25]:
ren_dict = {}
for item in res.keys():
    if item[0] == ' ':
        ren_dict[item] = item[1:]
res.rename(columns=ren_dict, inplace=True)

In [26]:
res['Region'] = 'China'

In [27]:
cat_dic = {'Number of Irrigated Areas over 10 000 Mu  (set)': 'Irrigated Areas',
           '33 000  Hectares and Over': 'Irrigated Areas',
           '20 000-33 000  Hectares': 'Irrigated Areas',
           'Effective Irrigated Area  (10 000 hectares)': 'Effective Irrigated Area',
           '33 000 Hectares and Over': 'Effective Irrigated Area',
           '20 000-33 000 Hectares': 'Effective Irrigated Area',
           'Number of Reservoirs  (unit)': 'Reservoirs',
           'Large Reservoir': 'Reservoirs',
           'Medium-sized Reservoir': 'Reservoirs',
           'Small Reservoir': 'Reservoirs',
           'Capacity of Reservoirs (100 million cu.m)': 'Capacity of Reservoirs',
           'Cap  Large Reservoir': 'Capacity of Reservoirs',
           'Cap  Medium-sized Reservoir': 'Capacity of Reservoirs',
           'Cap  Small Reservoir': 'Capacity of Reservoirs'}
unit_dic = {'Number of Irrigated Areas over 10 000 Mu  (set)': 'Number',
           '33 000  Hectares and Over': 'Number',
           '20 000-33 000  Hectares': 'Number',
           'Effective Irrigated Area  (10 000 hectares)': 'Units',
           '33 000 Hectares and Over': 'Units',
           '20 000-33 000 Hectares': 'Units',
           'Number of Reservoirs  (unit)': 'Units',
           'Large Reservoir': 'Units',
           'Medium-sized Reservoir': 'Units',
           'Small Reservoir': 'Units',
           'Capacity of Reservoirs (100 million cu.m)': 'Capacity of Reservoirs',
           'Cap  Large Reservoir': '100m m3',
           'Cap  Medium-sized Reservoir': '100m m3',
           'Cap  Small Reservoir': '100m m3'
           }
norm_res = norm_df(res, cat_dic, unit_dic, filename, tablename)

In [28]:
norm_res.replace({'indicator': {'Area with Flood Prevention Measures(10 000 hectares)': 'Area with Flood Prevention Measures',
       'Area of Land Protected by Dikes(10 000 hectares)': 'Area of Land Protected by Dikes'},
                          'category_1': {'Capacity of Reservoirs': 'Total',
                                       'Cap  Large Reservoir': 'Large Reservoir',
       'Cap  Medium-sized Reservoir': 'Medium-sized Reservoir', 'Cap  Small Reservoir': 'Small Reservoir'}}, inplace=True)

In [29]:
check_norm(norm_res)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    114 non-null    object 
 1   table_name   114 non-null    object 
 2   indicator    114 non-null    object 
 3   category_1   84 non-null     object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       114 non-null    object 
 7   region_type  114 non-null    object 
 8   Year         114 non-null    int64  
 9   value        114 non-null    float64
 10  units        114 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 9.9+ KB
None


,indicator,category_1,units,size
0,Irrigated Areas,Number of Irrigated Areas over 10 000 Mu (set),Number,6
1,Irrigated Areas,33 000 Hectares and Over,Number,6
2,Irrigated Areas,20 000-33 000 Hectares,Number,6
3,Effective Irrigated Area,Effective Irrigated Area (10 000 hectares),Units,6
4,Effective Irrigated Area,33 000 Hectares and Over,Units,6
5,Effective Irrigated Area,20 000-33 000 Hectares,Units,6
6,Reservoirs,Number of Reservoirs (unit),Units,6
7,Reservoirs,Large Reservoir,Units,6
8,Reservoirs,Medium-sized Reservoir,Units,6
9,Reservoirs,Small Reservoir,Units,6


In [30]:
dataframe_list['res'] = norm_res

### Buildings Dataset

In [31]:
filename = file_list[1]
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

['14-23  建筑业企业房屋建筑面积', '14-23  Floor Space of Buildings']

In [32]:
tablename = sheets[1]
build = read_sheet(dataset, tablename)

In [33]:
#Let's avoid repeated headers
for col in range(len(build.keys())):
    if str(build[build.keys()[col]][1]) == 'nan':
        build.iloc[1][build.keys()[col]] = build[build.keys()[col-1]][1]

In [34]:
build = clean_headers(build, 5)
build = year_region(build)
build = build.dropna(axis=0, subset=['Region'])
build.reset_index(inplace=True, drop=True)

In [35]:
build.dropna(axis=0, subset=['Year'], inplace=True)

In [36]:
cat_dic = {'Floor Space of Buildings Constructed Floor Space under Construction': ['Building Floorspace', 'Total', 'Under Construction'],
       'Floor Space of Buildings Constructed Floor Space Completed': ['Building Floorspace', 'Total', 'Completed'],
       'State-owned Floor Space under Construction': ['Building Floorspace', 'State-Owned', 'Under Construction'],
       'State-owned Floor Space Completed': ['Building Floorspace', 'State-Owned', 'Completed'],
       'Collective-owned Floor Space under Construction': ['Building Floorspace', 'Collective-Owned', 'Under Construction'],
       '(10 000 sq.m) Collective-owned Floor Space Completed':['Building Floorspace', 'Collective-Owned', 'Completed'],
           }
unit_dic = {'Floor Space of Buildings Constructed Floor Space under Construction': '10,000 sq.m',
       'Floor Space of Buildings Constructed Floor Space Completed': '10,000 sq.m',
       'State-owned Floor Space under Construction': '10,000 sq.m',
       'State-owned Floor Space Completed': '10,000 sq.m',
       'Collective-owned Floor Space under Construction': '10,000 sq.m',
       '(10 000 sq.m) Collective-owned Floor Space Completed': '10,000 sq.m'
           }
norm_build = norm_df(build, cat_dic, unit_dic, filename, tablename)

In [37]:
check_norm(norm_build)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    300 non-null    object 
 1   table_name   300 non-null    object 
 2   indicator    300 non-null    object 
 3   category_1   300 non-null    object 
 4   category_2   300 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       300 non-null    object 
 7   region_type  300 non-null    object 
 8   Year         300 non-null    int64  
 9   value        300 non-null    float64
 10  units        300 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 25.9+ KB
None


,indicator,category_1,category_2,units,size
0,Building Floorspace,Total,Under Construction,"10,000 sq.m",50
1,Building Floorspace,Total,Completed,"10,000 sq.m",50
2,Building Floorspace,State-Owned,Under Construction,"10,000 sq.m",50
3,Building Floorspace,State-Owned,Completed,"10,000 sq.m",50
4,Building Floorspace,Collective-Owned,Under Construction,"10,000 sq.m",50
5,Building Floorspace,Collective-Owned,Completed,"10,000 sq.m",50


In [38]:
dataframe_list['build'] = norm_build

### Employment Dataset

In [39]:
filename = file_list[2]
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

['4-2  按三次产业分就业人员数 (年底数)',
 '4-2  Number of Employed Persons',
 '1-13  就业基本情况',
 '1-14  分城乡就业人员年末人数',
 '1-15  分产业就业人员年末人数',
 '1-19  分行业城镇非私营单位就业人员年末人数']

In [40]:
tablename = sheets[1]
emp = read_sheet(dataset, tablename)

In [41]:
#Let's avoid repeated headers
for col in range(1, 5):
    if str(emp[emp.keys()[col]][2]) == 'nan':
        emp.iloc[2][emp.keys()[col]] = emp[emp.keys()[col-1]][2]

In [42]:
emp = clean_headers(emp, 6)
emp = emp.dropna(axis=0, subset=['Year'])

In [43]:
emp['Region'] = 'China'
emp.head()

,Year,Total Employed Persons (10 000 persons),Employed Primary Industry,Employed Secondary Industry,Employed Tertiary Industry,Composition in Percentage Primary Industry,Secondary Industry,Tertiary Industry,Region
0,1952,20729,17317,1531,1881,83.5,7.4,9.1,China
1,1957,23771,19309,2142,2320,81.2,9,9.8,China
2,1962,25910,21276,2059,2575,82.1,8,9.9,China
3,1965,28670,23396,2408,2866,81.6,8.4,10,China
4,1970,34432,27811,3518,3103,80.8,10.2,9,China


In [44]:
emp.keys()

Index(['Year', 'Total Employed Persons (10 000 persons)',
       'Employed Primary Industry', 'Employed Secondary Industry',
       'Employed Tertiary Industry',
       'Composition in Percentage Primary Industry', 'Secondary Industry',
       'Tertiary Industry', 'Region'],
      dtype='object', name=0)

In [45]:
cat_dic = {'Total Employed Persons (10 000 persons)': ['Employment', 'Total'],
           'Employed Primary Industry': ['Employment', 'Primary Industry'],
           'Employed Secondary Industry': ['Employment', 'Secondary Industry'],
           'Employed Tertiary Industry': ['Employment', 'Tertiary Industry'],
           'Composition in Percentage Primary Industry': ['Employment', 'Primary Industry'],
           'Secondary Industry': ['Employment', 'Secondary Industry'],
           'Tertiary Industry': ['Employment', 'Terciary Industry']}
unit_dic = {'Total Employed Persons (10 000 persons)': '10 000 persons',
           'Employed Primary Industry': '10 000 persons',
           'Employed Secondary Industry': '10 000 persons',
           'Employed Tertiary Industry': '10 000 persons',
           'Composition in Percentage Primary Industry': 'Percentage',
           'Secondary Industry': 'Percentage',
           'Tertiary Industry': 'Percentage'}
norm_emp = norm_df(emp, cat_dic, unit_dic, filename, tablename)

In [46]:
check_norm(norm_emp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    336 non-null    object 
 1   table_name   336 non-null    object 
 2   indicator    336 non-null    object 
 3   category_1   336 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       336 non-null    object 
 7   region_type  336 non-null    object 
 8   Year         336 non-null    int64  
 9   value        336 non-null    float64
 10  units        336 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 29.0+ KB
None


,indicator,category_1,units,size
0,Employment,Total,10 000 persons,48
1,Employment,Primary Industry,10 000 persons,48
2,Employment,Secondary Industry,10 000 persons,48
3,Employment,Tertiary Industry,10 000 persons,48
4,Employment,Primary Industry,Percentage,48
5,Employment,Secondary Industry,Percentage,48
6,Employment,Terciary Industry,Percentage,48


In [47]:
dataframe_list['emp'] = norm_emp

In [48]:
tablename = sheets[2]
forc = read_sheet(dataset, tablename)

In [49]:
for row in range(5, 8):
    forc.at[row, forc.keys()[1]] = f"Num {forc.iloc[row][forc.keys()[1]]}"
for row in range(27, 29):
    forc.at[row, forc.keys()[1]] = f"Rural {forc.iloc[row][forc.keys()[1]]}"

In [50]:
forc.drop(columns=[forc.keys()[0]], inplace=True)

In [51]:
forc = forc.transpose()
forc.reset_index(inplace=True, drop=True)

In [52]:
forc = clean_headers(forc, 1)

In [53]:
forc.dropna(axis=1, how='all', inplace=True)

In [54]:
forc.rename(columns={forc.keys()[0]: 'Year'}, inplace=True)
forc['Year'] = pd.to_numeric(forc['Year'])

In [55]:
forc['Region'] = 'China'

In [56]:
#Remove spaces in col names
ren_dict = {}
for item in forc.keys():
    if item.find('    ') > -1:
        ren_dict[item]= item.replace('    ', '')
    if item.find('   ') > -1:
        ren_dict[item]= item.replace('   ', '')
forc.rename(columns=ren_dict, inplace=True)

In [57]:
ren_dict = {}
for item in forc.keys():
    if item[0] == ' ':
        ren_dict[item] = item[1:]
forc.rename(columns=ren_dict, inplace=True)

In [58]:
cat_dic = {' Labour Force (10 000 persons)': ['Labour Force', 'Total'],
           ' Total Number of Employed Persons  ': ['Employed persons', 'Total'],
           'Num Primary Industry': ['Employed persons', 'Industry breakdown', 'Primary Industry'],
           'Num Secondary Industry': ['Employed persons', 'Industry breakdown', 'Secondary Industry'],
           'Num Tertiary Industry': ['Employed persons', 'Industry breakdown', 'Tertiary Industry'],
           'Primary Industry': ['Employed persons', 'Industry breakdown', 'Primary Industry'],
           'Secondary Industry': ['Employed persons', 'Industry breakdown', 'Secondary Industry'],
           'Tertiary Industry': ['Employed persons', 'Industry breakdown', 'Tertiary Industry'],
           ' Urban Employed Persons': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Total'],
           'State-owned Units': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'State-owned units'],
           'Urban Collective-owned Units': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Collective-owned units'],
           'Cooperative Units': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Cooperative units'],
           'Joint Ownership Units': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Joint ownership units'],
           'Limited Liability Corporations': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Limited Liability Corporations'],
           'Share-holding Corporations Ltd.': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Share-holding Corporations Ltd.'],
           'Private Enterprises': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Private Enterprises'],
           'Units with Funds from Hong Kong, Macao & ': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Units with Funds from Hong Kong, Macao & Taiwan'],
           'Foreign Funded Units': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Foreign Funded Units'],
           'Self-employed Individuals': ['Employed persons', 'Urban/Rural breakdown', 'Urban', 'Self-employed Individuals'],
           ' Rural Employed Persons': ['Employed persons', 'Urban/Rural breakdown', 'Rural', 'Total'],
           'Rural Private Enterprises': ['Employed persons', 'Urban/Rural breakdown', 'Rural', 'Private Enterprises'],
           'Rural Self-employed Individuals': ['Employed persons', 'Urban/Rural breakdown', 'Rural', 'Self-employed Individuals'],
           ' Number of Registered Unemployed Persons': ['Unemployed persons in Urban areas'],
           ' Registered Unemployment Rate in Urban': ['Unemployed persons in Urban areas']}

unit_dic = {' Labour Force (10 000 persons)': '10 000 persons',
            ' Total Number of Employed Persons  ': '10 000 persons',
           'Num Primary Industry': '10 000 persons',
           'Num Secondary Industry': '10 000 persons',
           'Num Tertiary Industry': '10 000 persons',
           'Primary Industry': 'Percentage',
           'Secondary Industry': 'Percentage',
           'Tertiary Industry': 'Percentage',
           ' Urban Employed Persons': '10 000 persons',
           'State-owned Units': '10 000 persons',
           'Urban Collective-owned Units': '10 000 persons',
           'Cooperative Units': '10 000 persons',
           'Joint Ownership Units': '10 000 persons',
           'Limited Liability Corporations': '10 000 persons',
           'Share-holding Corporations Ltd.': '10 000 persons',
           'Private Enterprises': '10 000 persons',
           'Units with Funds from Hong Kong, Macao & ': '10 000 persons',
           'Foreign Funded Units': '10 000 persons',
           'Self-employed Individuals': '10 000 persons',
           ' Rural Employed Persons': '10 000 persons',
           'Rural Private Enterprises': '10 000 persons',
           'Rural Self-employed Individuals': '10 000 persons',
           ' Number of Registered Unemployed Persons': '10 000 persons',
           ' Registered Unemployment Rate in Urban': 'Percentage'}
norm_forc = norm_df(forc, cat_dic, unit_dic, filename, tablename)

In [59]:
check_norm(norm_forc)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    216 non-null    object 
 1   table_name   216 non-null    object 
 2   indicator    216 non-null    object 
 3   category_1   198 non-null    object 
 4   category_2   180 non-null    object 
 5   category_3   126 non-null    object 
 6   Region       216 non-null    object 
 7   region_type  216 non-null    object 
 8   Year         216 non-null    float64
 9   value        216 non-null    float64
 10  units        216 non-null    object 
dtypes: float64(2), object(9)
memory usage: 18.7+ KB
None


,indicator,category_1,category_2,category_3,units,size
0,Labour Force,Total,NaN,NaN,10 000 persons,9
1,Employed persons,Total,NaN,NaN,10 000 persons,9
2,Employed persons,Industry breakdown,Primary Industry,NaN,10 000 persons,9
3,Employed persons,Industry breakdown,Secondary Industry,NaN,10 000 persons,9
4,Employed persons,Industry breakdown,Tertiary Industry,NaN,10 000 persons,9
5,Employed persons,Industry breakdown,Primary Industry,NaN,Percentage,9
6,Employed persons,Industry breakdown,Secondary Industry,NaN,Percentage,9
7,Employed persons,Industry breakdown,Tertiary Industry,NaN,Percentage,9
8,Employed persons,Urban/Rural breakdown,Urban,Total,10 000 persons,9
9,Employed persons,Urban/Rural breakdown,Urban,State-owned units,10 000 persons,9


In [60]:
dataframe_list['forc'] = norm_forc

In [61]:
tablename = sheets[3]
emp2 = read_sheet(dataset, tablename)

In [62]:
emp2 = clean_headers(emp2, 5)

In [63]:
emp2 = year_region(emp2, emp2.keys()[0])
emp2.rename(columns={'Number of Employed Persons at Year-end in Urban and Rural Areas 单位: 万人，% 年  份\n\nYear': 'Year',}, inplace=True)

In [64]:
cat_dic = {           '就业人员 Total Number of Employed Persons': ['Employment', 'Total'],
           '城镇        Urban 就业人员 Employed Persons': ['Employment', 'Urban'],
           '比重 Proportion': ['Employment', 'Urban'],
           '乡村        Rural 就业人员 Employed Persons': ['Employment', 'Rural'],
           '(10 000 persons,%) 比重 Proportion': ['Employment', 'Rural']}
           
unit_dic = {
           '就业人员 Total Number of Employed Persons': '10 000 persons',
           '城镇        Urban 就业人员 Employed Persons': '10 000 persons',
           '比重 Proportion': 'Percentage',
           '乡村        Rural 就业人员 Employed Persons': '10 000 persons',
           '(10 000 persons,%) 比重 Proportion': 'Percentage'}
norm_emp2 = norm_df(emp2, cat_dic, unit_dic, filename, tablename)

In [65]:
check_norm(norm_emp2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    340 non-null    object 
 1   table_name   340 non-null    object 
 2   indicator    340 non-null    object 
 3   category_1   340 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       340 non-null    object 
 7   region_type  340 non-null    object 
 8   Year         340 non-null    int64  
 9   value        340 non-null    float64
 10  units        340 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 29.3+ KB
None


,indicator,category_1,units,size
0,Employment,Total,10 000 persons,68
1,Employment,Urban,10 000 persons,68
2,Employment,Urban,Percentage,68
3,Employment,Rural,10 000 persons,68
4,Employment,Rural,Percentage,68


In [66]:
dataframe_list['emp2'] = norm_emp2

In [67]:
###### Skipping tablename=sheets[4] as the data is included in a previous table

In [68]:
tablename = sheets[5]
urb = read_sheet(dataset, tablename)

In [69]:
urb.drop(columns=urb.keys()[0], inplace=True)

In [70]:
urb = urb.transpose()
urb.reset_index(inplace=True, drop=True)

In [71]:
urb = clean_headers(urb, 1)

In [72]:
urb.dropna(axis=1, how='all', inplace=True)
urb.drop(columns=[urb.keys()[0]], inplace=True)
urb.rename(columns={urb.keys()[0]: 'Year'}, inplace=True)

In [73]:
urb = year_region(urb, 'Year')

In [74]:
cat_dic = {}
unit_dic = {}
for item in urb.keys():
    if item not in ['Year', 'Region']:
        if item == 'Production and Supply of Electricity, Heat,':
            label = item + 'Gas, and Water'
        elif item == 'Information Transmission, Software and Information':
            label = item + ' Technology'
        elif item == 'Management of Water Conservancy, Environment and':
            label = item + ' Public Facilities'
        elif item == 'Public Management, Social Security and Social':
            label = item + ' Organization'
        else:
            label = item
            
        cat_dic[item] = ['Employed People', 'Urban', label]
        unit_dic[item] = '10 000 persons'

In [75]:
norm_urb = norm_df(urb, cat_dic, unit_dic, filename, tablename)

In [76]:
check_norm(norm_urb)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    320 non-null    object 
 1   table_name   320 non-null    object 
 2   indicator    320 non-null    object 
 3   category_1   320 non-null    object 
 4   category_2   320 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       320 non-null    object 
 7   region_type  320 non-null    object 
 8   Year         320 non-null    float64
 9   value        320 non-null    float64
 10  units        320 non-null    object 
dtypes: float64(2), object(9)
memory usage: 27.6+ KB
None


,indicator,category_1,category_2,units,size
0,Employed People,Urban,Total,10 000 persons,16
1,Employed People,Urban,"Agriculture, Forestry, Animal Husbandry and Fi...",10 000 persons,16
2,Employed People,Urban,Mining,10 000 persons,16
3,Employed People,Urban,Manufacturing,10 000 persons,16
4,Employed People,Urban,"Production and Supply of Electricity, Heat,Gas...",10 000 persons,16
5,Employed People,Urban,Construction,10 000 persons,16
6,Employed People,Urban,Wholesale and Retail Trades,10 000 persons,16
7,Employed People,Urban,"Transport, Storage and Post",10 000 persons,16
8,Employed People,Urban,Hotels and Catering Services,10 000 persons,16
9,Employed People,Urban,"Information Transmission, Software and Informa...",10 000 persons,16


In [77]:
dataframe_list['urb'] = norm_urb

### Exchange rate

In [78]:
filename = file_list[3]
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

['18-8  人民币汇率 (年平均价)', '18-8  Exchange Rate of Renminbi']

In [79]:
tablename = sheets[1]
exc = read_sheet(dataset, tablename)

In [80]:
exc = clean_headers(exc, 2)

In [81]:
exc = year_region(exc, 'Year')
exc['Region'] = None

In [82]:
exc.rename(columns={'(RMB yuan) 100 Euros': '100 Euros'}, inplace=True)

In [83]:
cat_dic = {}
unit_dic = {}
for item in exc.keys():
    if item not in ['Year', 'Region']:
        label = item.split('100 ')[1]
        cat_dic[item] = ['Exchange Rate', label]
        unit_dic[item] = item
norm_exc = norm_df(exc, cat_dic, unit_dic, filename, tablename)

In [84]:
norm_exc['region_type'] = None

In [85]:
check_norm(norm_exc)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    140 non-null    object 
 1   table_name   140 non-null    object 
 2   indicator    140 non-null    object 
 3   category_1   140 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       0 non-null      object 
 7   region_type  0 non-null      object 
 8   Year         140 non-null    int64  
 9   value        123 non-null    float64
 10  units        140 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 12.2+ KB
None


,indicator,category_1,units,size
0,Exchange Rate,US Dollars,100 US Dollars,35
1,Exchange Rate,Japanese Yen,100 Japanese Yen,35
2,Exchange Rate,Hong Kong Dollars,100 Hong Kong Dollars,35
3,Exchange Rate,Euros,100 Euros,35


In [86]:
dataframe_list['exc'] = norm_exc

### GDP

In [87]:
filename = file_list[4]
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

['3-1  国内生产总值',
 '3-2  国内生产总值构成',
 '3-3  不变价国内生产总值',
 '3-4  国内生产总值指数',
 '3-5  国内生产总值指数',
 '3-6  分行业增加值',
 '3-7  三次产业和主要行业贡献率',
 '3-8  三次产业和主要行业对国内生产总值增长的拉动',
 '3-9  地区生产总值 (2019年)',
 '3-10  支出法国内生产总值',
 '3-11  支出法国内生产总值及构成',
 '3-12  实际最终消费及构成',
 '3-13  居民消费水平',
 '3-1  Gross Domestic Product',
 '3-2  Composition of Gross Domes',
 '3-3  Gross Domestic Product at ',
 '3-4  Indices of Gross Domestic ',
 '3-5  Indices of Gross Domestic ',
 '3-6  Value-added by Sector',
 '3-7  Share of the Contributions',
 '3-8  Contribution of the Three ',
 '3-9  Gross Regional Product (20',
 '3-10  Gross Domestic Product by',
 '3-11  Components of Gross Domes',
 '3-12  Actual Final Consumption ',
 '3-13  Household Consumption Exp',
 'Total Factor Productivity']

In [88]:
tablename = sheets[13]
gdpabs = read_sheet(dataset, tablename)

In [89]:
gdpabs = clean_headers(gdpabs, 7)

In [90]:
gdpabs.rename(columns={gdpabs.keys()[0]: 'Year'}, inplace=True)

In [91]:
gdpabs = year_region(gdpabs, 'Year')
#gdpabs['Region'] = None

In [92]:
cat_dic = {}
unit_dic = {}
for item in gdpabs.keys():
    if item in ['Year', 'Region']:
        pass
    elif item in gdpabs.keys()[1:3]:
        unit_dic[item] = '100 million yuan'
    elif item in gdpabs.keys()[3:6]:
        cat_dic[item] = ['GDP', 'Industry breakdown', item]
        unit_dic[item] = '100 million yuan'
    else:
        cat_dic[item] = ['GDP', 'Sector', item]
        unit_dic[item] = '100 million yuan'
norm_gdpabs = norm_df(gdpabs, cat_dic, unit_dic, filename, tablename)

In [93]:
check_norm(norm_gdpabs)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    704 non-null    object 
 1   table_name   704 non-null    object 
 2   indicator    704 non-null    object 
 3   category_1   616 non-null    object 
 4   category_2   616 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       704 non-null    object 
 7   region_type  704 non-null    object 
 8   Year         704 non-null    int64  
 9   value        672 non-null    float64
 10  units        704 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 60.6+ KB
None


,indicator,category_1,category_2,units,size
0,Gross National Income,NaN,NaN,100 million yuan,44
1,Gross Domestic Product,NaN,NaN,100 million yuan,44
2,GDP,Industry breakdown,Primary Industry,100 million yuan,44
3,GDP,Industry breakdown,Secondary Industry,100 million yuan,44
4,GDP,Industry breakdown,Tertiary Industry,100 million yuan,44
5,GDP,Sector,"Agriculture, Forestry, Animal Husbandry and Fi...",100 million yuan,44
6,GDP,Sector,Industry,100 million yuan,44
7,GDP,Sector,Construction,100 million yuan,44
8,GDP,Sector,Wholesale and Retail Trades,100 million yuan,44
9,GDP,Sector,"Transport, Storage and Post",100 million yuan,44


In [94]:
dataframe_list['gdpabs'] = norm_gdpabs

In [95]:
tablename = sheets[14]
gdpper = read_sheet(dataset, tablename)

In [96]:
gdpper = clean_headers(gdpper, 6)

In [97]:
gdpper.rename(columns={gdpper.keys()[0]: 'Year', 'Gross Domestic Product': 'GDP', '(%) Others': 'Others'}, inplace=True)

In [98]:
gdpper = year_region(gdpper, 'Year')

In [99]:
cat_dic = {}
unit_dic = {}
for item in gdpper.keys():
    if item in ['Year', 'Region']:
        pass
    elif item in gdpper.keys()[1]:
        unit_dic[item] = 'Percentage'
    elif item in gdpper.keys()[2:5]:
        cat_dic[item] = ['GDP', 'Industry breakdown', item]
        unit_dic[item] = 'Percentage'
    else:
        cat_dic[item] = ['GDP', 'Sector', item]
        unit_dic[item] = 'Percentage'
norm_gdpper = norm_df(gdpper, cat_dic, unit_dic, filename, tablename)

In [100]:
check_norm(norm_gdpper)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    546 non-null    object 
 1   table_name   546 non-null    object 
 2   indicator    546 non-null    object 
 3   category_1   504 non-null    object 
 4   category_2   504 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       546 non-null    object 
 7   region_type  546 non-null    object 
 8   Year         546 non-null    int64  
 9   value        546 non-null    float64
 10  units        546 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 47.0+ KB
None


,indicator,category_1,category_2,units,size
0,GDP,NaN,NaN,Percentage,42
1,GDP,Industry breakdown,Primary Industry,Percentage,42
2,GDP,Industry breakdown,Secondary Industry,Percentage,42
3,GDP,Industry breakdown,Tertiary Industry,Percentage,42
4,GDP,Sector,"Agriculture, Forestry, Animal Husbandry and Fi...",Percentage,42
5,GDP,Sector,Industry,Percentage,42
6,GDP,Sector,Construction,Percentage,42
7,GDP,Sector,Wholesale and Retail Trades,Percentage,42
8,GDP,Sector,"Transport, Storage and Post",Percentage,42
9,GDP,Sector,Hotels and Catering Services,Percentage,42


In [101]:
dataframe_list['gdpper'] = norm_gdpper

In [102]:
tablename = sheets[15]
gdpcon = read_sheet(dataset, tablename)

In [103]:
gdpcon.drop(axis=0, index=[61, 62], inplace=True)

In [104]:
gdpcon = clean_headers(gdpcon, 7)

In [105]:
gdpcon.rename(columns={'Gross Domestic Product Price Base Year=1970': 'GDP (constant price)', '(100 million yuan) Others': 'Others'}, inplace=True)

In [106]:
gdpcon = year_region(gdpcon, 'Year')
gdpcon.dropna(axis=0, subset=['Year'], inplace=True)

In [107]:
gdpcon.keys()

Index(['Year', 'GDP (constant price)', 'Primary Industry',
       'Secondary Industry', 'Tertiary Industry',
       'Agriculture, Forestry, Animal Husbandry and Fishery', 'Industry',
       'Construction', 'Wholesale and Retail Trades',
       'Transport, Storage and Post', 'Hotels and Catering Services',
       'Financial Intermediation', 'Real Estate', 'Others', 'Region'],
      dtype='object', name=0)

In [108]:
cat_dic = {}
unit_dic = {}
unit = '100 million yuan'
for item in gdpcon.keys():
    if item in ['Year', 'Region']:
        pass
    elif item in gdpcon.keys()[1]:
        unit_dic[item] = unit
    elif item in gdpcon.keys()[2:5]:
        cat_dic[item] = ['GDP (constant price)', 'Industry breakdown', item]
        unit_dic[item] = unit
    else:
        cat_dic[item] = ['GDP (constant price)', 'Sector', item]
        unit_dic[item] = unit
norm_gdpcon = norm_df(gdpcon, cat_dic, unit_dic, filename, tablename)

In [109]:
check_norm(norm_gdpcon)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    624 non-null    object 
 1   table_name   624 non-null    object 
 2   indicator    624 non-null    object 
 3   category_1   576 non-null    object 
 4   category_2   576 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       624 non-null    object 
 7   region_type  624 non-null    object 
 8   Year         624 non-null    int64  
 9   value        624 non-null    float64
 10  units        624 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 53.8+ KB
None


,indicator,category_1,category_2,units,size
0,GDP (constant price),NaN,NaN,100 million yuan,48
1,GDP (constant price),Industry breakdown,Primary Industry,100 million yuan,48
2,GDP (constant price),Industry breakdown,Secondary Industry,100 million yuan,48
3,GDP (constant price),Industry breakdown,Tertiary Industry,100 million yuan,48
4,GDP (constant price),Sector,"Agriculture, Forestry, Animal Husbandry and Fi...",100 million yuan,48
5,GDP (constant price),Sector,Industry,100 million yuan,48
6,GDP (constant price),Sector,Construction,100 million yuan,48
7,GDP (constant price),Sector,Wholesale and Retail Trades,100 million yuan,48
8,GDP (constant price),Sector,"Transport, Storage and Post",100 million yuan,48
9,GDP (constant price),Sector,Hotels and Catering Services,100 million yuan,48


In [110]:
dataframe_list['gdpcon'] = norm_gdpcon

In [111]:
tablename = sheets[16]
gdpind = read_sheet(dataset, tablename)

In [112]:
gdpind = clean_headers(gdpind, 7)

In [113]:
gdpind.rename(columns={gdpind.keys()[0]: 'Year', '(preceding year=100) Per Capita GNI': 'GNI per capita',
                       'Per Capita GDP': 'GDP per capita'}, inplace=True)

In [114]:
gdpind = year_region(gdpind, 'Year')

In [115]:
gdpind.keys()

Index(['Year', 'Gross National Income', 'Gross Domestic Product',
       'Primary Industry', 'Secondary Industry', 'Tertiary Industry',
       'Agriculture, Forestry, Animal Husbandry and Fishery', 'Industry',
       'Construction', 'Wholesale and Retail Trades',
       'Transport, Storage and Post', 'Hotels and Catering Services',
       'Financial Intermediation', 'Real Estate', 'Others', 'GDP per capita',
       'GNI per capita', 'Region'],
      dtype='object', name=0)

In [116]:
cat_dic = {}
unit_dic = {}
unit = 'Percentage'
for item in gdpind.keys():
    if item in ['Year', 'Region']:
        pass
    elif item in ['Gross National Income', 'Gross Domestic Product',
       'GDP per capita', 'GNI per capita']:
        label = item + ' (constant price)'
        cat_dic[item] = [label]
        unit_dic[item] = unit
    elif item in gdpind.keys()[3:6]:
        cat_dic[item] = ['GDP (constant price)', 'Industry breakdown', item]
        unit_dic[item] = unit
    else:
        cat_dic[item] = ['GDP (constant price)', 'Sector', item]
        unit_dic[item] = unit
norm_gdpind = norm_df(gdpind, cat_dic, unit_dic, filename, tablename)

In [117]:
check_norm(norm_gdpind)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    672 non-null    object 
 1   table_name   672 non-null    object 
 2   indicator    672 non-null    object 
 3   category_1   504 non-null    object 
 4   category_2   504 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       672 non-null    object 
 7   region_type  672 non-null    object 
 8   Year         672 non-null    int64  
 9   value        672 non-null    float64
 10  units        672 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 57.9+ KB
None


,indicator,category_1,category_2,units,size
0,Gross National Income (constant price),NaN,NaN,Percentage,42
1,Gross Domestic Product (constant price),NaN,NaN,Percentage,42
2,GDP (constant price),Industry breakdown,Primary Industry,Percentage,42
3,GDP (constant price),Industry breakdown,Secondary Industry,Percentage,42
4,GDP (constant price),Industry breakdown,Tertiary Industry,Percentage,42
5,GDP (constant price),Sector,"Agriculture, Forestry, Animal Husbandry and Fi...",Percentage,42
6,GDP (constant price),Sector,Industry,Percentage,42
7,GDP (constant price),Sector,Construction,Percentage,42
8,GDP (constant price),Sector,Wholesale and Retail Trades,Percentage,42
9,GDP (constant price),Sector,"Transport, Storage and Post",Percentage,42


In [118]:
dataframe_list['gdpind'] = norm_gdpind

In [119]:
tablename = sheets[17]
gdp78 = read_sheet(dataset, tablename)

In [120]:
gdp78 = clean_headers(gdp78, 7)

In [121]:
gdp78.rename(columns={gdp78.keys()[0]: 'Year', '(year of 1978=100) Per Capita GNI': 'GNI per capita',
                       'Per Capita GDP': 'GDP per capita'}, inplace=True)

In [122]:
gdp78 = year_region(gdp78, 'Year')

In [123]:
cat_dic = {}
unit_dic = {}
unit = 'Percentage'
for item in gdp78.keys():
    if item in ['Year', 'Region']:
        pass
    elif item in ['Gross National Income', 'Gross Domestic Product',
       'GDP per capita', 'GNI per capita']:
        label = item + ' (compared to 1978)'
        cat_dic[item] = [label]
        unit_dic[item] = unit
    elif item in gdp78.keys()[3:6]:
        cat_dic[item] = ['GDP (compared to 1978)', 'Industry breakdown', item]
        unit_dic[item] = unit
    else:
        cat_dic[item] = ['GDP (compared to 1978)', 'Sector', item]
        unit_dic[item] = unit
norm_gdp78 = norm_df(gdp78, cat_dic, unit_dic, filename, tablename)

In [124]:
check_norm(norm_gdp78)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    672 non-null    object 
 1   table_name   672 non-null    object 
 2   indicator    672 non-null    object 
 3   category_1   504 non-null    object 
 4   category_2   504 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       672 non-null    object 
 7   region_type  672 non-null    object 
 8   Year         672 non-null    int64  
 9   value        672 non-null    float64
 10  units        672 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 57.9+ KB
None


,indicator,category_1,category_2,units,size
0,Gross National Income (compared to 1978),NaN,NaN,Percentage,42
1,Gross Domestic Product (compared to 1978),NaN,NaN,Percentage,42
2,GDP (compared to 1978),Industry breakdown,Primary Industry,Percentage,42
3,GDP (compared to 1978),Industry breakdown,Secondary Industry,Percentage,42
4,GDP (compared to 1978),Industry breakdown,Tertiary Industry,Percentage,42
5,GDP (compared to 1978),Sector,"Agriculture, Forestry, Animal Husbandry and Fi...",Percentage,42
6,GDP (compared to 1978),Sector,Industry,Percentage,42
7,GDP (compared to 1978),Sector,Construction,Percentage,42
8,GDP (compared to 1978),Sector,Wholesale and Retail Trades,Percentage,42
9,GDP (compared to 1978),Sector,"Transport, Storage and Post",Percentage,42


In [125]:
dataframe_list['gdp78'] = norm_gdp78

In [126]:
tablename = sheets[18]
gdpval = read_sheet(dataset, tablename)

In [127]:
gdpval = gdpval.transpose()
gdpval.reset_index(inplace=True, drop=True)

In [128]:
gdpval = clean_headers(gdpval, 1)

In [129]:
gdpval.drop(columns=[gdpval.keys()[0]], inplace=True)
gdpval.drop(columns=[gdpval.keys()[0]], inplace=True)

In [130]:
ren_col = {'Sector': 'Year'}
col_drop = []
for col in range(len(gdpval.keys())):
    if gdpval[list(gdpval.keys())[col]].isnull().values.all(axis=0):
        ren_col[gdpval.keys()[col-1]] = gdpval.keys()[col-1] + gdpval.keys()[col]
        col_drop.append(gdpval.keys()[col])
gdpval.rename(columns=ren_col, inplace=True)
gdpval.drop(columns=col_drop, inplace=True)

In [131]:
gdpval

,Year,Gross Domestic Product,"Agriculture, Forestry, Animal Husbandry and Fishery",Mining,Manufacturing,"Production and Supply of Electricity, Heat, Gas and Water",Construction,Wholesale and Retail Trades,"Transport, Storage and Post",Hotels and Catering Services,...,Financial Intermediation,Real Estate,Leasing and Business Services,Scientific Research and Technical Services,"Management of Water Conservancy, Environment and Public Facilities","Household Service, Repair and Other Services",Education,Health and Social Service,"Culture, Sports and Entertainment","Public Management, Social Security and Social Organization"
0,2015.0,688858.218049,59852.632998,19243.124596,199435.962044,16289.823256,47761.338341,67719.574952,30519.505893,12306.111158,...,56299.847886,42573.815095,18089.729489,13887.189059,4128.58912,10586.738446,24370.450368,14765.372001,5063.802239,28447.801134
1,2016.0,746395.059484,62451.028611,18514.845414,209508.930087,17382.660339,51498.930933,73724.452734,33028.694098,13607.799208,...,59963.976943,49969.395424,21528.646554,15394.22992,4354.607657,12183.509747,26964.885175,16766.766056,5579.809436,33847.772153
2,2017.0,832035.94856,64660.040276,21380.081927,233876.459529,19862.712087,57905.557514,81156.606483,37121.874758,15056.044402,...,64844.303637,57085.954986,25273.339465,17444.692422,4602.567319,13725.447705,30208.447047,18548.752372,6625.386172,38848.785383
3,2018,919281.129067,67558.747209,22592.34662,255937.161684,22559.841571,65493.037085,88903.732183,40337.247413,16520.606068,...,70610.25629,64622.99215,29468.518536,20175.299264,5096.075402,14793.279697,34001.413132,20652.613885,7301.342411,43923.121024


In [132]:
gdpval = year_region(gdpval, 'Year')

In [133]:
gdpval.keys()

Index(['Year', 'Gross Domestic Product',
       'Agriculture, Forestry, Animal Husbandry and Fishery', 'Mining',
       'Manufacturing',
       'Production and Supply of Electricity, Heat,   Gas and Water',
       'Construction', 'Wholesale and Retail Trades',
       'Transport, Storage and Post', 'Hotels and Catering Services',
       'Information Transmission, Software and  Information Technology',
       'Financial Intermediation', 'Real Estate',
       'Leasing and Business Services',
       'Scientific Research and Technical Services',
       'Management of Water Conservancy, Environment  and Public Facilities',
       'Household Service, Repair and Other Services', 'Education',
       'Health and Social Service', 'Culture, Sports and Entertainment',
       'Public Management, Social Security and   Social Organization',
       'Region'],
      dtype='object', name=0)

In [134]:
cat_dic = {}
unit_dic = {}
unit = '100 million yuan'
for item in gdpval.keys():
    if item in ['Year', 'Region']:
        pass
    elif item == 'Gross Domestic Product':
        label = 'GDP (value-added)'
        cat_dic[item] = [label]
        unit_dic[item] = unit
    else:
        cat_dic[item] = ['GDP (value-added)', item]
        unit_dic[item] = unit
norm_gdpval = norm_df(gdpval, cat_dic, unit_dic, filename, tablename)

In [135]:
check_norm(norm_gdpval)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    80 non-null     object 
 1   table_name   80 non-null     object 
 2   indicator    80 non-null     object 
 3   category_1   76 non-null     object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       80 non-null     object 
 7   region_type  80 non-null     object 
 8   Year         80 non-null     float64
 9   value        80 non-null     float64
 10  units        80 non-null     object 
dtypes: float64(2), object(9)
memory usage: 7.0+ KB
None


,indicator,category_1,units,size
0,GDP (value-added),NaN,100 million yuan,4
1,GDP (value-added),"Agriculture, Forestry, Animal Husbandry and Fi...",100 million yuan,4
2,GDP (value-added),Mining,100 million yuan,4
3,GDP (value-added),Manufacturing,100 million yuan,4
4,GDP (value-added),"Production and Supply of Electricity, Heat, ...",100 million yuan,4
5,GDP (value-added),Construction,100 million yuan,4
6,GDP (value-added),Wholesale and Retail Trades,100 million yuan,4
7,GDP (value-added),"Transport, Storage and Post",100 million yuan,4
8,GDP (value-added),Hotels and Catering Services,100 million yuan,4
9,GDP (value-added),"Information Transmission, Software and Inform...",100 million yuan,4


In [136]:
dataframe_list['gdpval'] = norm_gdpval

In [137]:
tablename = sheets[19]
gdpmain = read_sheet(dataset, tablename)

In [138]:
gdpmain = clean_headers(gdpmain, 6)

In [139]:
gdpmain.rename(columns={gdpmain.keys()[0]: 'Year', '(%) Financial Intermediation': 'Financial Intermediation'}, inplace=True)

In [140]:
gdpmain.drop(axis=0, index=[42, 43], inplace=True)

In [141]:
gdpmain = year_region(gdpmain, 'Year')

In [142]:
gdpmain.drop(columns=['Gross Domestic Product'], inplace=True)

In [143]:
gdpmain.keys()

Index(['Year', 'Primary Industry', 'Secondary Industry', 'Tertiary Industry',
       'Industry', 'Wholesale and Retail Trades', 'Financial Intermediation',
       'Region'],
      dtype='object', name=0)

In [144]:
cat_dic = {}
unit_dic = {}
unit = 'Percentage'
for item in gdpmain.keys():
    if item in ['Year', 'Region']:
        pass
    elif item in gdpmain.keys()[1:4]:
        cat_dic[item] = ['GDP (main contributors to increase)', 'Industry breakdown', item]
        unit_dic[item] = unit
    else:
        cat_dic[item] = ['GDP (main contributors to increase)', 'Sector', item]
        unit_dic[item] = unit
norm_gdpmain = norm_df(gdpmain, cat_dic, unit_dic, filename, tablename)

In [145]:
check_norm(norm_gdpmain)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    252 non-null    object 
 1   table_name   252 non-null    object 
 2   indicator    252 non-null    object 
 3   category_1   252 non-null    object 
 4   category_2   252 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       252 non-null    object 
 7   region_type  252 non-null    object 
 8   Year         252 non-null    int64  
 9   value        252 non-null    float64
 10  units        252 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 21.8+ KB
None


,indicator,category_1,category_2,units,size
0,GDP (main contributors to increase),Industry breakdown,Primary Industry,Percentage,42
1,GDP (main contributors to increase),Industry breakdown,Secondary Industry,Percentage,42
2,GDP (main contributors to increase),Industry breakdown,Tertiary Industry,Percentage,42
3,GDP (main contributors to increase),Sector,Industry,Percentage,42
4,GDP (main contributors to increase),Sector,Wholesale and Retail Trades,Percentage,42
5,GDP (main contributors to increase),Sector,Financial Intermediation,Percentage,42


In [146]:
dataframe_list['gdpmain'] = norm_gdpmain

In [147]:
tablename = sheets[20]
gdpmaing = read_sheet(dataset, tablename)

In [148]:
gdpmaing = clean_headers(gdpmaing, 6)

In [149]:
gdpmaing.rename(columns={gdpmaing.keys()[0]: 'Year', '(percentage points)   Financial Intermediation': 'Financial Intermediation'}, inplace=True)

In [150]:
gdpmaing.drop(axis=0, index=[42, 43], inplace=True)

In [151]:
gdpmaing = year_region(gdpmaing, 'Year')

In [152]:
cat_dic = {}
unit_dic = {}
unit = 'Percentage'
for item in gdpmaing.keys():
    if item in ['Year', 'Region']:
        pass
    elif item == 'Gross Domestic Product':
        cat_dic[item] = ['GDP (main contributors to growth)']
        unit_dic[item] = unit
    elif item in gdpmaing.keys()[2:5]:
        cat_dic[item] = ['GDP (main contributors to growth)', 'Industry breakdown', item]
        unit_dic[item] = unit
    else:
        cat_dic[item] = ['GDP (main contributors to growth)', 'Sector', item]
        unit_dic[item] = unit
norm_gdpmaing = norm_df(gdpmaing, cat_dic, unit_dic, filename, tablename)

In [153]:
check_norm(norm_gdpmaing)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    294 non-null    object 
 1   table_name   294 non-null    object 
 2   indicator    294 non-null    object 
 3   category_1   252 non-null    object 
 4   category_2   252 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       294 non-null    object 
 7   region_type  294 non-null    object 
 8   Year         294 non-null    int64  
 9   value        294 non-null    float64
 10  units        294 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 25.4+ KB
None


,indicator,category_1,category_2,units,size
0,GDP (main contributors to growth),NaN,NaN,Percentage,42
1,GDP (main contributors to growth),Industry breakdown,Primary Industry,Percentage,42
2,GDP (main contributors to growth),Industry breakdown,Secondary Industry,Percentage,42
3,GDP (main contributors to growth),Industry breakdown,Tertiary Industry,Percentage,42
4,GDP (main contributors to growth),Sector,Industry,Percentage,42
5,GDP (main contributors to growth),Sector,Wholesale and Retail Trades,Percentage,42
6,GDP (main contributors to growth),Sector,Financial Intermediation,Percentage,42


In [154]:
dataframe_list['gdpmaing'] = norm_gdpmaing

In [155]:
tablename = sheets[21]
print(tablename)
gdp19 = read_sheet(dataset, tablename)

3-9  Gross Regional Product (20


In [156]:
gdp19 = clean_headers(gdp19, 8)

In [157]:
#Separating into different datasets as we have repeated column names
gdp19m = gdp19.iloc[:,:15].copy() #cols in million yuan
gdp19p = gdp19.iloc[:,15:18].copy() #cols in percentage
gdp19p2 = gdp19.iloc[:,18:].copy() #cols in other percentage

In [158]:
gdp19m.rename(columns={gdp19m.keys()[0]: 'Region',
                      'Value-Added by Three Strata of Industry Primary Industry': 'Primary Industry',
                      'Per Capita Gross Regional Product (yuan)': 'Per Capita Gross Regional Product',
                       'Value-Added by Sector Agriculture,  Forestry, Animal Husbandry and Fishery': 'Agriculture, Forestry, Animal Husbandry and Fishery'
                      }, inplace=True)
gdp19p.rename(columns={'Composition (GRP=100) Primary Industry': 'Primary Industry -P',
                       'Secondary Industry': 'Secondary Industry -P',
                       'Tertiary Industry': 'Tertiary Industry -P'}, inplace=True)
gdp19p2.rename(columns={'(100 million yuan) Per Capita Gross Regional Product': 'Per Capita Gross Regional Product -PY',
                     'Indices (preceding year=100) Gross Regional Product': 'Gross Regional Product -PY',
                       'Primary Industry': 'Primary Industry -PY',
                       'Secondary Industry': 'Secondary Industry -PY',
                       'Tertiary Industry': 'Tertiary Industry -PY'}, inplace=True)

In [159]:
gdp19 = pd.concat([gdp19m, gdp19p, gdp19p2], axis=1)

In [160]:
gdp19.drop(axis=0, index=[31], inplace=True)
gdp19.dropna(axis=0, how='all', inplace=True)

In [161]:
gdp19['Year'] = 2019

In [162]:
cat_dic = {}
unit_dic = {}
for item in gdp19.keys():
    if item in ['Year', 'Region']:
        pass
    elif item in gdp19.keys()[1:15]:
        unit = '100 million yuan'
        if item in ['Gross Regional Product', 'Per Capita Gross Regional Product']:
            cat_dic[item] = [item]
        elif item in gdpmaing.keys()[2:5]:
            cat_dic[item] = ['GRP', 'Industry breakdown', item]
        else:
            cat_dic[item] = ['GRP', 'Sector', item]
        unit_dic[item] = unit
    elif item in gdp19.keys()[15:18]:
        cat_dic[item] = ['GRP', 'Industry breakdown', item.split(' -')[0]]
        unit_dic[item] = 'Percentage'
    elif item in gdp19.keys()[18:-1]:
        try:
            label = item.split(' -')[0]
        except:
            label = item
        if label in ['Gross Regional Product', 'Per Capita Gross Regional Product']:
            cat_dic[item] = [label+' (compared to preceding year)']
        else:
            cat_dic[item] = ['GRP (compared to preceding year)', 'Industry breakdown', label]
        unit_dic[item] = 'Percentage'
norm_gdp19 = norm_df(gdp19, cat_dic, unit_dic, filename, tablename)

In [163]:
check_norm(norm_gdp19)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    682 non-null    object 
 1   table_name   682 non-null    object 
 2   indicator    682 non-null    object 
 3   category_1   558 non-null    object 
 4   category_2   558 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       682 non-null    object 
 7   region_type  682 non-null    object 
 8   Year         682 non-null    int64  
 9   value        682 non-null    float64
 10  units        682 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 58.7+ KB
None


,indicator,category_1,category_2,units,size
0,Gross Regional Product,NaN,NaN,100 million yuan,31
1,GRP,Industry breakdown,Primary Industry,100 million yuan,31
2,GRP,Industry breakdown,Secondary Industry,100 million yuan,31
3,GRP,Industry breakdown,Tertiary Industry,100 million yuan,31
4,GRP,Sector,"Agriculture, Forestry, Animal Husbandry and Fi...",100 million yuan,31
5,GRP,Sector,Industry,100 million yuan,31
6,GRP,Sector,Construction,100 million yuan,31
7,GRP,Sector,Wholesale and Retail Trades,100 million yuan,31
8,GRP,Sector,"Transport, Storage and Post",100 million yuan,31
9,GRP,Sector,Hotels and Catering Services,100 million yuan,31


In [164]:
dataframe_list['gdp19'] = norm_gdp19

In [165]:
tablename = sheets[22]
print(tablename)
gdpexp = read_sheet(dataset, tablename)

3-10  Gross Domestic Product by


In [166]:
gdpexp.dropna(axis=1, how='all', inplace=True)

In [167]:
gdpexp = clean_headers(gdpexp, 6)

In [168]:
gdpexp.rename(columns={gdpexp.keys()[0]: 'Year',
                        'Gross Domestic Product by Expenditure  Approach (100 million yuan) Gross Domestic Product by Expenditure  Approach (100 million yuan)':
                               'Gross Domestic Product by Expenditure Approach',
                        'Final Consumption Rate (%) Final Consumption Rate (%)': 'Final Consumption Rate',
                       'Capital Formation Rate (%) Capital Formation Rate (%)': 'Capital Formation Rate'
                       }, inplace=True)

In [169]:
gdpexp.drop(axis=0, index=[42, 43, 44, 45], inplace=True)

In [170]:
gdpexp = year_region(gdpexp, 'Year')

In [171]:
gdpexp.keys()

Index(['Year',
       'Gross Domestic Product by Expenditure  Approach (100 million yuan)',
       'Final Consumption Expenditure', 'Gross Capital Formation',
       'Net Exports of Goods and Services', 'Final Consumption Rate (%)',
       'Capital Formation Rate (%)', 'Region'],
      dtype='object', name=0)

In [172]:
cat_dic = {}
unit_dic = {}

for item in gdpexp.keys():
    if item in ['Year', 'Region']:
        pass
    elif item == 'Gross Domestic Product by Expenditure Approach':
        cat_dic[item] = [item, 'Total']
        unit_dic[item] = '100 million yuan'
    elif item in ['Final Consumption Expenditure Final Consumption Expenditure',
       'Gross Capital Formation Gross Capital Formation',
       'Net Exports of Goods and Services Net Exports of Goods and Services']:
        cat_dic[item] = ['Gross Domestic Product by Expenditure Approach', item]
        unit_dic[item] = '100 million yuan'
    else:
        cat_dic[item] = [item]
        unit_dic[item] = 'Percentage'
norm_gdpexp = norm_df(gdpexp, cat_dic, unit_dic, filename, tablename)

In [173]:
check_norm(norm_gdpexp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    252 non-null    object 
 1   table_name   252 non-null    object 
 2   indicator    252 non-null    object 
 3   category_1   0 non-null      object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       252 non-null    object 
 7   region_type  252 non-null    object 
 8   Year         252 non-null    int64  
 9   value        252 non-null    float64
 10  units        252 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 21.8+ KB
None


,indicator,category_1,units,size
0,Gross Domestic Product by Expenditure Approac...,NaN,Percentage,42
1,Final Consumption Expenditure,NaN,Percentage,42
2,Gross Capital Formation,NaN,Percentage,42
3,Net Exports of Goods and Services,NaN,Percentage,42
4,Final Consumption Rate (%),NaN,Percentage,42
5,Capital Formation Rate (%),NaN,Percentage,42


In [174]:
dataframe_list['gdpexp'] = norm_gdpexp

In [175]:
tablename = sheets[23]
print(tablename)
gdpexp2 = read_sheet(dataset, tablename)

3-11  Components of Gross Domes


In [176]:
gdpexp2 = clean_headers(gdpexp2, 9)

In [177]:
gdpexp2.dropna(axis=0, how='all', inplace=True)

In [178]:
#Separating into different datasets as we have repeated column names
gdpexp2a = gdpexp2.iloc[:,:9].copy()
gdpexp2b = gdpexp2.iloc[:,9:-2].copy()

In [179]:
gdpexp2a.rename(columns={gdpexp2a.keys()[0]: 'Year',
                        'Final Consumption Expenditure (100 million yuan) Household Consumption Expenditure': 'Household Consumption Expenditure',
                        'Gross Capital Formation (100 million yuan) Gross Fixed Capital Formation': 'Gross Fixed Capital Formation',
                        'Net Exports of Goods and  Services (100 million yuan) Exports': 'Exports'}, inplace=True)
gdpexp2b.rename(columns={'Final Consumption Expenditure=100 Household Consumption Expenditure': 'Household Consumption Expenditure-F',
                       'Government Consumption Expenditure': 'Government Consumption Expenditure -F',
                       'Household Consumption Expenditure=100 Urban Households': 'Urban Households -F',
                       'Rural Households': 'Rural Households -F',
                       'Gross Capital Formation=100 Gross Fixed Capital Formation': 'Gross Fixed Capital Formation -F',
                       'Change in Inventories': 'Change in Inventories -F'
                        }, inplace=True)

In [180]:
gdpexp2 = pd.concat([gdpexp2a, gdpexp2b], axis=1)

In [181]:
gdpexp2 = year_region(gdpexp2, 'Year')

In [182]:
gdpexp2.keys()

Index(['Year', 'Household Consumption Expenditure', 'Urban Households',
       'Rural Households', 'Government Consumption Expenditure',
       'Gross Fixed Capital Formation', 'Change in Inventories', 'Exports',
       'Imports', 'Household Consumption Expenditure-F',
       'Government Consumption Expenditure -F', 'Urban Households -F',
       'Rural Households -F', 'Gross Fixed Capital Formation -F',
       'Change in Inventories -F', 'Region'],
      dtype='object', name=0)

In [183]:
cat_dic = {'Household Consumption Expenditure': ['Consumption Expenditure', 'Household Consumption Expenditure', 'Total'],
           'Urban Households': ['Consumption Expenditure', 'Household Consumption Expenditure', 'Urban Households'],
           'Rural Households': ['Consumption Expenditure', 'Household Consumption Expenditure', 'Rural Households'],
           'Government Consumption Expenditure': ['Consumption Expenditure', 'Government Consumption Expenditure'],
           'Gross Fixed Capital Formation': ['Gross Capital Formation', 'Gross Fixed Capital Formation'],
           'Change in Inventories': ['Gross Capital Formation', 'Change in Inventories'],
           'Exports': ['Trade of Goods and Services', 'Exports'],
           'Imports': ['Trade of Goods and Services', 'Imports'],
           'Household Consumption Expenditure-F': ['Consumption Expenditure', 'Household Consumption Expenditure', 'Total'],
           'Government Consumption Expenditure -F': ['Consumption Expenditure', 'Government Consumption Expenditure'],
           'Urban Households -F': ['Consumption Expenditure', 'Household Consumption Expenditure', 'Urban Households'],
           'Rural Households -F': ['Consumption Expenditure', 'Household Consumption Expenditure', 'Rural Households'],
           'Gross Fixed Capital Formation -F': ['Gross Capital Formation', 'Gross Fixed Capital Formation'],
           'Change in Inventories -F': ['Gross Capital Formation', 'Change in Inventories']}
unit_dic = {}

for item in gdpexp2.keys():
    if item in ['Year', 'Region']:
        pass
    elif item in gdpexp2.keys()[1:9]:
        unit_dic[item] = '100 million yuan'
    else:
        unit_dic[item] = 'Percentage'

norm_gdpexp2 = norm_df(gdpexp2, cat_dic, unit_dic, filename, tablename)

In [184]:
check_norm(norm_gdpexp2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    588 non-null    object 
 1   table_name   588 non-null    object 
 2   indicator    588 non-null    object 
 3   category_1   588 non-null    object 
 4   category_2   252 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       588 non-null    object 
 7   region_type  588 non-null    object 
 8   Year         588 non-null    int64  
 9   value        512 non-null    float64
 10  units        588 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 50.7+ KB
None


,indicator,category_1,category_2,units,size
0,Consumption Expenditure,Household Consumption Expenditure,Total,100 million yuan,42
1,Consumption Expenditure,Household Consumption Expenditure,Urban Households,100 million yuan,42
2,Consumption Expenditure,Household Consumption Expenditure,Rural Households,100 million yuan,42
3,Consumption Expenditure,Government Consumption Expenditure,NaN,100 million yuan,42
4,Gross Capital Formation,Gross Fixed Capital Formation,NaN,100 million yuan,42
5,Gross Capital Formation,Change in Inventories,NaN,100 million yuan,42
6,Trade of Goods and Services,Exports,NaN,100 million yuan,42
7,Trade of Goods and Services,Imports,NaN,100 million yuan,42
8,Consumption Expenditure,Household Consumption Expenditure,Total,Percentage,42
9,Consumption Expenditure,Government Consumption Expenditure,NaN,Percentage,42


In [185]:
dataframe_list['gdpexp2'] = norm_gdpexp2

In [186]:
tablename = sheets[24]
cons = read_sheet(dataset, tablename)

In [187]:
cons = clean_headers(cons, 5)

In [188]:
cons_a = cons.iloc[:,:3].copy()
cons_b = cons.iloc[:,3:].copy()

In [189]:
cons_a.rename(columns={cons_a.keys()[0]: 'Year',
                     ' Actual Final Consumption  (100 million yuan) Household Actual  Final Consumption': 'Household Actual Final Consumption'}, inplace=True)
cons_b.rename(columns={'  Composition  (Actual Final Consumption=100) Household Actual  Final Consumption': 'Household Actual Final Consumption -P',
                      'Government Actual  Final Consumption': 'Government Actual  Final Consumption -P'}, inplace=True)

In [190]:
cons = pd.concat([cons_a, cons_b], axis=1)

In [191]:
cons = year_region(cons, 'Year')

In [192]:
cat_dic = {}
unit_dic = {}
for item in cons.keys():
    if item in ['Year', 'Region']:
        pass
    elif item.find(' -P') > -1:
        cat_dic[item] = ['Actual Final Consumption', item.split(' -')[0]]
        unit_dic[item] = 'Percentage'
    else:
        cat_dic[item] = ['Actual Final Consumption', item]
        unit_dic[item] = '100 million yuan'
norm_cons = norm_df(cons, cat_dic, unit_dic, filename, tablename)

In [193]:
check_norm(norm_cons)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    164 non-null    object 
 1   table_name   164 non-null    object 
 2   indicator    164 non-null    object 
 3   category_1   164 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       164 non-null    object 
 7   region_type  164 non-null    object 
 8   Year         164 non-null    int64  
 9   value        164 non-null    float64
 10  units        164 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 14.2+ KB
None


,indicator,category_1,units,size
0,Actual Final Consumption,Household Actual Final Consumption,100 million yuan,41
1,Actual Final Consumption,Government Actual Final Consumption,100 million yuan,41
2,Actual Final Consumption,Household Actual Final Consumption,Percentage,41
3,Actual Final Consumption,Government Actual Final Consumption,Percentage,41


In [194]:
dataframe_list['cons'] = norm_cons

In [195]:
tablename = sheets[25]
conse = read_sheet(dataset, tablename)

In [196]:
for col in range(5, 8):
    conse.iloc[0, col] = 'P-'
for col in range(8, 11):
    conse.iloc[0, col] = '78-'

In [197]:
conse = clean_headers(conse, 6)

In [198]:
conse.rename(columns={conse.keys()[0]: 'Year'}, inplace=True)

In [199]:
conse = year_region(conse, 'Year')
conse['Region'] = 'China'

In [200]:
conse.keys()

Index(['Year', 'Value (yuan) All Households', 'Urban Households',
       'Rural Households', 'Urban/Rural Consumption Ratio (Rural Household=1)',
       'P- Index (Preceding Year=100) All Households', 'P- Urban Households',
       'P- Rural Households', '78- Index (1978=100) All Households',
       '78- Urban Households', '78- Rural Households', 'Region'],
      dtype='object', name=0)

In [201]:
cat_dic = {}
unit_dic = {}
for item in range(len(conse.keys())):
    if conse.keys()[item] in ['Year', 'Region']:
        pass
    elif item in range(1, 5):
        try:
            label = conse.keys()[item].split(') ')[1]
        except:
            label = conse.keys()[item]
        cat_dic[conse.keys()[item]] = ['Consumption expenditure', label]
        unit_dic[conse.keys()[item]] = 'Yuan'
    elif item in range(5, 8):
        try:
            label = conse.keys()[item].split(') ')[1]
        except:
            label = conse.keys()[item].split('P- ')[1]
        cat_dic[conse.keys()[item]] = ['Consumption expenditure', label]
        unit_dic[conse.keys()[item]] = 'Percentage'
    elif item in range(8, 11):
        try:
            label = conse.keys()[item].split(') ')[1]
        except:
            label = conse.keys()[item].split('78- ')[1]
        cat_dic[conse.keys()[item]] = ['Consumption expenditure (compared to 1978)', label]
        unit_dic[conse.keys()[item]] = 'Percentage'
norm_conse = norm_df(conse, cat_dic, unit_dic, filename, tablename)

In [202]:
check_norm(norm_conse)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    290 non-null    object 
 1   table_name   290 non-null    object 
 2   indicator    290 non-null    object 
 3   category_1   290 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       290 non-null    object 
 7   region_type  290 non-null    object 
 8   Year         290 non-null    int64  
 9   value        250 non-null    float64
 10  units        290 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 25.0+ KB
None


,indicator,category_1,units,size
0,Consumption expenditure,All Households,Yuan,29
1,Consumption expenditure,Urban Households,Yuan,29
2,Consumption expenditure,Rural Households,Yuan,29
3,Consumption expenditure,Urban/Rural Consumption Ratio (Rural Household=1),Yuan,29
4,Consumption expenditure,All Households,Percentage,29
5,Consumption expenditure,Urban Households,Percentage,29
6,Consumption expenditure,Rural Households,Percentage,29
7,Consumption expenditure (compared to 1978),All Households,Percentage,29
8,Consumption expenditure (compared to 1978),Urban Households,Percentage,29
9,Consumption expenditure (compared to 1978),Rural Households,Percentage,29


In [203]:
dataframe_list['conse'] = norm_conse

In [204]:
tablename = sheets[26]
print(tablename)
tfp = read_sheet(dataset, tablename)

Total Factor Productivity


In [205]:
tfp['Year'] = pd.DatetimeIndex(tfp['DATE']).year

In [206]:
tfp = tfp[['Year', 'Total Factor Productivity at Constant National Prices for China']]

In [207]:
tfp.rename(columns={'Total Factor Productivity at Constant National Prices for China': 'Total Factor Productivity (Constant National Prices, 2011)'}, inplace=True)


In [208]:
tfp['Region'] = 'China'

In [209]:
cat_dic = {'Total Factor Productivity (Constant National Prices, 2011)': ['Total Factor Productivity (Constant National Prices, 2011)']}
unit_dic = {'Total Factor Productivity (Constant National Prices, 2011)': None}
norm_tfp = norm_df(tfp, cat_dic, unit_dic, filename, tablename)

In [210]:
check_norm(norm_tfp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    62 non-null     object 
 1   table_name   62 non-null     object 
 2   indicator    62 non-null     object 
 3   category_1   0 non-null      object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       62 non-null     object 
 7   region_type  62 non-null     object 
 8   Year         62 non-null     int64  
 9   value        62 non-null     float64
 10  units        0 non-null      object 
dtypes: float64(1), int64(1), object(9)
memory usage: 5.5+ KB
None


,indicator,category_1,units,size
0,Total Factor Productivity (Constant National P...,NaN,NaN,62


In [211]:
dataframe_list['tfp'] = norm_tfp

### Imports and exports

In [212]:
filename = file_list[5]
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

['11-1  对外经济贸易基本情况',
 '11-2  货物进出口总额',
 '11-3  按国际贸易标准分类分进出口商品金额(2019年)',
 '11-5  我国同各国(地区)海关货物进出口总额(2019年)',
 '11-6  出口主要货物数量和金额(2019年)',
 '11-7  进口主要货物数量和金额(2019年)',
 '11-1  International Trade and E',
 '11-2  International Trade in Go',
 '11-3  International Trade in Go',
 '11-5  International Trade in Go',
 '11-6  Major Exported Commoditie',
 '11-7  Major Imported Commoditie']

In [213]:
tablename = sheets[6]
print(tablename)
ecoo = read_sheet(dataset, tablename)

11-1  International Trade and E


In [214]:
#Remove indicators that are in the next dataset
ecoo.drop(axis=0, index=[x for x in range(1, 11)], inplace=True)
ecoo.reset_index(inplace=True, drop=True)

In [215]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = []
for row in ecoo.index:
    if str(ecoo[ecoo.keys()[1]][row]) == 'nan':
        ecoo.iloc[row-1, 0] = ecoo[ecoo.keys()[0]][row-1] + ecoo[ecoo.keys()[0]][row]
        to_del.append(row)
ecoo.drop(axis=0, index=to_del, inplace=True)
ecoo.reset_index(inplace=True, drop=True)

In [216]:
#get ready for transposing
ren_dict = {}
for col, num in zip(ecoo.keys(), range(len(ecoo.keys()))):
    ren_dict[col] = num
ecoo.rename(columns=ren_dict, inplace=True)

In [217]:
#remove extra spaces in indicator names
for row in ecoo.index:
    new_label = re.sub("\s\s+", " ", ecoo[ecoo.keys()[0]][row])
    if new_label[0] == ' ':
        ecoo.at[row, ecoo.keys()[0]] = new_label[1:]
    else:
        ecoo.at[row, ecoo.keys()[0]] =  new_label

In [218]:
ecoo[ecoo.keys()[0]].unique()

array(['Item', 'Import and Export of Services (USD 100 million)',
       'Exports', 'Imports', 'Balance',
       'Inward Foreign Direct Investments (USD 100 million)',
       'Newly Established Foreign-Invested Enterprises (FIEs) (unit)',
       'Number of Registered Enterprises (household)',
       'Total Investment (USD 100 million)',
       'Registered Capital (USD 100 million)',
       'Capital from Foreign Investors',
       'Outbound Direct Investment (USD 100 million) Economic Cooperation with Foreign',
       'Contracted Value of Contracted Projects',
       'Value of Turnover Fulfilled of Contracted Projects'], dtype=object)

In [219]:
ecoo = ecoo.transpose()

In [220]:
#add indicators as header names
header = ecoo.iloc[0] #grab the first row for the header
ecoo = ecoo[1:].copy() #take the data less the header row
ecoo.columns = header #set the header row as the df header
ecoo.reset_index(inplace=True, drop=True)

In [221]:
ecoo.rename(columns={'Item': 'Year'}, inplace=True)

In [222]:
ecoo.keys()

Index(['Year', 'Import and Export of Services (USD 100 million)', 'Exports',
       'Imports', 'Balance',
       'Inward Foreign Direct Investments (USD 100 million)',
       'Newly Established Foreign-Invested Enterprises (FIEs) (unit)',
       'Number of Registered Enterprises (household)',
       'Total Investment (USD 100 million)',
       'Registered Capital (USD 100 million)',
       'Capital from Foreign Investors',
       'Outbound Direct Investment (USD 100 million) Economic Cooperation with Foreign',
       'Contracted Value of Contracted Projects',
       'Value of Turnover Fulfilled of Contracted Projects'],
      dtype='object', name=0)

In [223]:
tablename = sheets[7]
print(tablename)
itrade = read_sheet(dataset, tablename)

11-2  International Trade in Go


In [224]:
for col in range(1, 5):
    itrade.iloc[0, col] = 'R-'
for col in range(5, 9):
    itrade.iloc[0, col] = 'U-'

In [225]:
itrade = clean_headers(itrade, 3)

In [226]:
itrade = year_region(itrade, 'Year')
itrade['Region'] = 'China'

In [227]:
itrade.keys()

Index(['Year', 'R- Total', 'R- Exports', 'R- Imports', 'R- Balance',
       'U- Total', 'U- Exports', 'U- Imports', 'U- Balance', 'Region'],
      dtype='object', name=0)

In [228]:
cat_dic = {}
unit_dic = {}
for item in range(len(itrade.keys())):
    if itrade.keys()[item] in ['Year', 'Region']:
        pass
    elif item in range(1, 5):
        label = itrade.keys()[item].split('- ')[1]
        cat_dic[itrade.keys()[item]] = ['International Trade of Goods', label]
        unit_dic[itrade.keys()[item]] = '100 million RMB'
    elif item in range(5, 9):
        label = itrade.keys()[item].split('- ')[1]
        cat_dic[itrade.keys()[item]] = ['International Trade of Goods', label]
        unit_dic[itrade.keys()[item]] = '100 million USD'
norm_itrade = norm_df(itrade, cat_dic, unit_dic, filename, tablename)

In [229]:
check_norm(norm_itrade)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    280 non-null    object 
 1   table_name   280 non-null    object 
 2   indicator    280 non-null    object 
 3   category_1   280 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       280 non-null    object 
 7   region_type  280 non-null    object 
 8   Year         280 non-null    int64  
 9   value        264 non-null    float64
 10  units        280 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 24.2+ KB
None


,indicator,category_1,units,size
0,International Trade of Goods,Total,100 million RMB,35
1,International Trade of Goods,Exports,100 million RMB,35
2,International Trade of Goods,Imports,100 million RMB,35
3,International Trade of Goods,Balance,100 million RMB,35
4,International Trade of Goods,Total,100 million USD,35
5,International Trade of Goods,Exports,100 million USD,35
6,International Trade of Goods,Imports,100 million USD,35
7,International Trade of Goods,Balance,100 million USD,35


In [230]:
dataframe_list['itrade'] = norm_itrade

In [231]:
tablename = sheets[8]
print(tablename)
sitc = read_sheet(dataset, tablename)

11-3  International Trade in Go


In [232]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = []
for row in sitc.index:
    if str(sitc[sitc.keys()[1]][row]) == 'nan':
        sitc.iloc[row-1, 0] = sitc[sitc.keys()[0]][row-1] + sitc[sitc.keys()[0]][row]
        to_del.append(row)
sitc.drop(axis=0, index=to_del, inplace=True)
sitc.reset_index(inplace=True, drop=True)

In [233]:
sitc.iloc[0, 2] = 'Exports'
sitc.iloc[0, 4] = 'Imports'

In [234]:
ren_dict = {}
for col, num in zip(sitc.keys(), range(len(sitc.keys()))):
    ren_dict[col] = num
sitc.rename(columns=ren_dict, inplace=True)

In [235]:
sitc = clean_headers(sitc, 3)

In [236]:
sitc.rename(columns={' Imports RMB 100 million': 'Imports RMB 100 million'}, inplace=True)
col_dict = {}
for col in range(1, 5):
    col_dict[col] = [sitc.keys()[col].split(' ')[0], sitc.keys()[col].split(' ')[1]]
col_dict

{1: ['Exports', 'RMB'],
 2: ['Exports', 'USD'],
 3: ['Imports', 'RMB'],
 4: ['Imports', 'USD']}

In [237]:
cats = []
for row in sitc.index:
    if sitc[sitc.keys()[0]][row][0] != ' ':
        cats.append(sitc[sitc.keys()[0]][row])
    else:
        sitc.at[row, sitc.keys()[0]] = re.sub("\s\s+", " ", sitc[sitc.keys()[0]][row])[1:]
cats

['Total',
 'Primary Goods',
 'Food and Live Animals',
 'Beverages and Tobacco',
 'Crude materials, Inedible, Except Fuels',
 'Mineral Fuels, Lubricants and Related Materials',
 'Animal and Vegetable Oils, Fats and Wax',
 'Manufactured Goods',
 'Chemicals and Related Products',
 'Manufactured Goods Classified Chiefly by Material',
 'Machinery and Transport Equipment',
 'Miscellaneous Manufactured Articles',
 'Commodities and Transactions not classified elsewhere   in the SITC']

In [238]:
norm_sitc = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])

for col in range(1, 5):
    for row in sitc.index:
        if sitc[sitc.keys()[0]][row] in cats:
            cat1 = sitc[sitc.keys()[0]][row]
            cat2 = 'Total'
        else:
            cat2 = sitc[sitc.keys()[0]][row]
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': f'{col_dict[col][0]} by SITC',
                    'category_1': cat1,
                    'category_2': cat2,
                    'category_3': None,
                    'Region': 'China',
                    'region_type': 'Country',
                    'Year': 2019,
                    'value': sitc.iloc[row, col],
                    'units': f'100 million {col_dict[col][1]}'}
        norm_sitc = norm_sitc.append(row_dict, ignore_index=True)
norm_sitc['Year'] = pd.to_numeric(norm_sitc['Year'])

In [239]:
check_norm(norm_sitc)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    304 non-null    object 
 1   table_name   304 non-null    object 
 2   indicator    304 non-null    object 
 3   category_1   304 non-null    object 
 4   category_2   304 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       304 non-null    object 
 7   region_type  304 non-null    object 
 8   Year         304 non-null    int64  
 9   value        304 non-null    float64
 10  units        304 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 26.2+ KB
None


,indicator,category_1,category_2,units,size
0,Exports by SITC,Total,Total,100 million RMB,1
1,Exports by SITC,Primary Goods,Total,100 million RMB,1
2,Exports by SITC,Food and Live Animals,Total,100 million RMB,1
3,Exports by SITC,Food and Live Animals,Live Animals,100 million RMB,1
4,Exports by SITC,Food and Live Animals,Meat and Meat Preparations,100 million RMB,1
...,...,...,...,...,...
295,Imports by SITC,Miscellaneous Manufactured Articles,Articles of Apparel and Clothing Accessories,100 million USD,1
296,Imports by SITC,Miscellaneous Manufactured Articles,Footwear,100 million USD,1
297,Imports by SITC,Miscellaneous Manufactured Articles,"Professional, Scientific and Controlling Instr...",100 million USD,1
298,Imports by SITC,Miscellaneous Manufactured Articles,"Photographic Apparatus, Equipment and Supplies...",100 million USD,1


In [240]:
dataframe_list['sitc'] = norm_sitc

In [241]:
tablename = sheets[9]
print(tablename)
trco = read_sheet(dataset, tablename)

11-5  International Trade in Go


In [242]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = []
for row in trco.index:
    if str(trco[trco.keys()[1]][row]) == 'nan':
        trco.iloc[row-1, 0] = trco[trco.keys()[0]][row-1] + trco[trco.keys()[0]][row]
        to_del.append(row)
trco.drop(axis=0, index=to_del, inplace=True)
trco.reset_index(inplace=True, drop=True)

In [243]:
for col in range(1, 4):
    trco.iloc[0, col] = 'R-'
for col in range(4, 7):
    trco.iloc[0, col] = 'U-'

In [244]:
trco = clean_headers(trco, 2)

In [245]:
cats = []
for row in trco.index:
    if trco[trco.keys()[0]][row][0] != ' ':
        cats.append(trco[trco.keys()[0]][row])
    elif trco[trco.keys()[0]][row][2] != ' ':
        label = re.sub("\s\s+", " ", trco[trco.keys()[0]][row])[1:]
        cats.append(label)
        trco.at[row, trco.keys()[0]] = label
        
    else:
        trco.at[row, trco.keys()[0]] = re.sub("\s\s+", " ", trco[trco.keys()[0]][row])[1:]
cats
cats = cats[1:]

In [246]:
norm_trco = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])

for col in range(1, 7):
    for row in trco.index:
        if trco[trco.keys()[0]][row].find('Other') > -1:
            rt = 'Other'
        elif trco[trco.keys()[0]][row] in cats:
            rt = 'Continent'
        elif trco[trco.keys()[0]][row] == 'Total':
            rt = 'Global'
        else:
            rt = 'Country'
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': 'International Trade (by region)',
                    'category_1': trco.keys()[col].split('- ')[1],
                    'category_2': cat2,
                    'category_3': None,
                    'Region': trco.iloc[row,0],
                    'region_type': rt,
                    'Year': 2019,
                    'value': trco.iloc[row, col],
                    'units': f"100 million {'RMD' if trco.keys()[col].split('- ')[0] == 'R' else 'USD'}"}
        norm_trco = norm_trco.append(row_dict, ignore_index=True)
norm_trco['Year'] = pd.to_numeric(norm_trco['Year'])

In [247]:
check_norm(norm_trco)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482 entries, 0 to 1481
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    1482 non-null   object 
 1   table_name   1482 non-null   object 
 2   indicator    1482 non-null   object 
 3   category_1   1482 non-null   object 
 4   category_2   1482 non-null   object 
 5   category_3   0 non-null      object 
 6   Region       1482 non-null   object 
 7   region_type  1482 non-null   object 
 8   Year         1482 non-null   int64  
 9   value        1480 non-null   float64
 10  units        1482 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 127.5+ KB
None


,indicator,category_1,category_2,units,size
0,International Trade (by region),Total,Total,100 million RMD,247
1,International Trade (by region),Exports,Total,100 million RMD,247
2,International Trade (by region),Imports,Total,100 million RMD,247
3,International Trade (by region),Total,Total,100 million USD,247
4,International Trade (by region),Exports,Total,100 million USD,247
5,International Trade (by region),Imports,Total,100 million USD,247


In [248]:
dataframe_list['trco'] = norm_trco

In [249]:
tablename = sheets[10]
print(tablename)
excom = read_sheet(dataset, tablename)

11-6  Major Exported Commoditie


In [250]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = []
for row in excom.index:
    if str(excom[excom.keys()[2]][row]) == 'nan':
        excom.iloc[row-1, 0] = excom[excom.keys()[0]][row-1] + excom[excom.keys()[0]][row]
        to_del.append(row)
excom.drop(axis=0, index=to_del, inplace=True)
excom.reset_index(inplace=True, drop=True)

In [251]:
excom = clean_headers(excom, 2)

In [252]:
cats = []
for row in excom.index:
    if excom[excom.keys()[0]][row][0] != ' ':
        excom.append(excom[excom.keys()[0]][row])
    elif excom[excom.keys()[0]][row][2] != ' ':
        label = re.sub("\s\s+", " ", excom[excom.keys()[0]][row])[1:]
        cats.append(label)
        excom.at[row, excom.keys()[0]] = label
        
    else:
        excom.at[row, excom.keys()[0]] = re.sub("\s\s+", " ", excom[excom.keys()[0]][row])[1:]
#cats

In [253]:
norm_excom = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])
unit_dic = {2: '10 000 RMB', 3: '10 000 USD'}
for col in range(1, 4):
    for row in excom.index:
        if excom.iloc[row,0] in cats:
            cat1 = excom.iloc[row,0].split(' (')[0]
            cat2 = None
        else:
            cat2 = excom.iloc[row,0].split(' (')[0]

        if col == 1:
            try:
                un =excom.iloc[row,0].split(' (')[1].split(')')[0]
            except:
                un = None
        else:
            un = unit_dic[col]
        
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': 'Exported Commodities',
                    'category_1': cat1,
                    'category_2': cat2,
                    'category_3': None,
                    'Region': 'China',
                    'region_type': 'Country',
                    'Year': 2019,
                    'value': excom.iloc[row, col],
                    'units': un}
        norm_excom = norm_excom.append(row_dict, ignore_index=True)
norm_excom['Year'] = pd.to_numeric(norm_excom['Year'])

In [254]:
check_norm(norm_excom)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    372 non-null    object 
 1   table_name   372 non-null    object 
 2   indicator    372 non-null    object 
 3   category_1   372 non-null    object 
 4   category_2   48 non-null     object 
 5   category_3   0 non-null      object 
 6   Region       372 non-null    object 
 7   region_type  372 non-null    object 
 8   Year         372 non-null    int64  
 9   value        357 non-null    float64
 10  units        354 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 32.1+ KB
None


,indicator,category_1,category_2,units,size
0,Exported Commodities,Live Hogs,NaN,ton,1
1,Exported Commodities,Live Poultry,NaN,ton,1
2,Exported Commodities,"Frozen, Fresh Beef",NaN,ton,1
3,Exported Commodities,"Frozen, Fresh Pork",NaN,ton,1
4,Exported Commodities,Frozen Chicken,NaN,ton,1
...,...,...,...,...,...
367,Exported Commodities,Rattan Products,NaN,10 000 USD,1
368,Exported Commodities,Straw Mats and Straw Products,NaN,10 000 USD,1
369,Exported Commodities,Wickerwork,NaN,10 000 USD,1
370,Exported Commodities,Mechanical and Electrical Products,NaN,10 000 USD,1


In [255]:
dataframe_list['excom'] = norm_excom

In [256]:
tablename = sheets[11]
print(tablename)
imcom = read_sheet(dataset, tablename)

11-7  Major Imported Commoditie


In [257]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = []
for row in imcom.index:
    if str(imcom[imcom.keys()[2]][row]) == 'nan':
        imcom.iloc[row-1, 0] = imcom[imcom.keys()[0]][row-1] + imcom[imcom.keys()[0]][row]
        to_del.append(row)
imcom.drop(axis=0, index=to_del, inplace=True)
imcom.reset_index(inplace=True, drop=True)

In [258]:
imcom = clean_headers(imcom, 2)

In [259]:
cats = []
for row in imcom.index:
    if imcom[imcom.keys()[0]][row][0] != ' ':
        imcom.append(imcom[imcom.keys()[0]][row])
    elif imcom[imcom.keys()[0]][row][2] != ' ':
        label = re.sub("\s\s+", " ", imcom[imcom.keys()[0]][row])[1:]
        cats.append(label)
        imcom.at[row, imcom.keys()[0]] = label
        
    else:
        imcom.at[row, imcom.keys()[0]] = re.sub("\s\s+", " ", imcom[imcom.keys()[0]][row])[1:]
#cats

In [260]:
norm_imcom = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])
unit_dic = {2: '10 000 RMB', 3: '10 000 USD'}
for col in range(1, 4):
    for row in imcom.index:
        if imcom.iloc[row,0] in cats:
            cat1 = imcom.iloc[row,0].split(' (')[0]
            cat2 = None
        else:
            cat2 = imcom.iloc[row,0].split(' (')[0]

        if col == 1:
            try:
                un =imcom.iloc[row,0].split(' (')[1].split(')')[0]
            except:
                un = None
        else:
            un = unit_dic[col]
        
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': 'Imported Commodities',
                    'category_1': cat1,
                    'category_2': cat2,
                    'category_3': None,
                    'Region': 'China',
                    'region_type': 'Country',
                    'Year': 2019,
                    'value': imcom.iloc[row, col],
                    'units': un}
        norm_imcom = norm_imcom.append(row_dict, ignore_index=True)
norm_imcom['Year'] = pd.to_numeric(norm_imcom['Year'])

In [261]:
check_norm(norm_imcom)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    186 non-null    object 
 1   table_name   186 non-null    object 
 2   indicator    186 non-null    object 
 3   category_1   186 non-null    object 
 4   category_2   33 non-null     object 
 5   category_3   0 non-null      object 
 6   Region       186 non-null    object 
 7   region_type  186 non-null    object 
 8   Year         186 non-null    int64  
 9   value        182 non-null    float64
 10  units        179 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 16.1+ KB
None


,indicator,category_1,category_2,units,size
0,Imported Commodities,Cereals and Cereals Flour,NaN,10 000 tons,1
1,Imported Commodities,Cereals and Cereals Flour,Wheat,10 000 tons,1
2,Imported Commodities,Cereals and Cereals Flour,Paddy and Rice,10 000 tons,1
3,Imported Commodities,Soybean,NaN,10 000 tons,1
4,Imported Commodities,Edible Vegetable Oil,NaN,10 000 tons,1
...,...,...,...,...,...
181,Imported Commodities,Chassis with Engines,NaN,10 000 USD,1
182,Imported Commodities,Aircraft and Others,NaN,10 000 USD,1
183,Imported Commodities,Medical Instruments and Appliances,NaN,10 000 USD,1
184,Imported Commodities,Mechanical and Electrical Products,NaN,10 000 USD,1


In [262]:
norm_imcom.replace({'category_1': {'Natural Rubber': 'Natural Rubber (including Latex)',
                                   'Synthetic Rubber': 'Synthetic Rubber (including Latex)',
                                   'Paper and Paperboard': 'Paper and Paperboard (Unchopped in Shape)'},
                    'units': {'including Latex': '10 000 tons', 'Unchopped in Shape': '10 000 tons'}},
                            inplace=True)

In [263]:
dataframe_list['imcom'] = norm_imcom

### Industry

In [264]:
filename = file_list[6]
print(filename)
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

Industry_工业


['13-12  工业产品产量', '13-12  Output of Industrial Pro']

In [265]:
tablename = sheets[1]
print(tablename)
indo = read_sheet(dataset, tablename)

13-12  Output of Industrial Pro


In [266]:
ren_cols = {}
hname = {0: 'Item', 1:'2018', 2:'2019'}
for col in range(0, 3):
    ren_cols[indo.keys()[col]]=hname[col]
indo.rename(columns=ren_cols, inplace=True)
del_cols = [0]
for row in range(91, 104):
    del_cols.append(row)
indo.drop(axis=0, index=del_cols, inplace=True)
indo.drop(axis=1, columns=indo.keys()[3:], inplace=True)
indo.reset_index(inplace=True, drop=True)

In [267]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = []
for row in indo.index:
    if str(indo[indo.keys()[2]][row]) == 'nan':
        indo.iloc[row-1, 0] = indo[indo.keys()[0]][row-1] + indo[indo.keys()[0]][row]
        to_del.append(row)
indo.drop(axis=0, index=to_del, inplace=True)
indo.reset_index(inplace=True, drop=True)

In [268]:
cats = []
for row in indo.index:
    if indo[indo.keys()[0]][row][0] != ' ':
        indo.append(imcom[indo.keys()[0]][row])
    elif indo[indo.keys()[0]][row][2] != ' ':
        label = re.sub("\s\s+", " ", indo[indo.keys()[0]][row])[1:]
        cats.append(label)
        indo.at[row, indo.keys()[0]] = label
        
    else:
        indo.at[row, indo.keys()[0]] = re.sub("\s\s+", " ", indo[indo.keys()[0]][row])[1:]
#cats

In [269]:
norm_indo = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])

for col in range(1, 3):
    for row in indo.index:
        if indo.iloc[row,0] in cats:
            cat1 = indo.iloc[row,0].split(' (')[0]
            cat2 = None
        else:
            cat2 = indo.iloc[row,0].split(' (')[0]
            
        try:
            un = indo.iloc[row,0].split(' (')[1].split(')')[0]
        except:
            un = indo.iloc[row,0].split('(')[1].split(')')[0]
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': 'Industrial Production',
                    'category_1': cat1,
                    'category_2': cat2,
                    'category_3': None,
                    'Region': 'China',
                    'region_type': 'Country',
                    'Year': int(indo.keys()[col]),
                    'value': indo.iloc[row, col],
                    'units': un}
        norm_indo = norm_indo.append(row_dict, ignore_index=True)
norm_indo['Year'] = pd.to_numeric(norm_indo['Year'])

In [270]:
check_norm(norm_indo)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    178 non-null    object 
 1   table_name   178 non-null    object 
 2   indicator    178 non-null    object 
 3   category_1   178 non-null    object 
 4   category_2   46 non-null     object 
 5   category_3   0 non-null      object 
 6   Region       178 non-null    object 
 7   region_type  178 non-null    object 
 8   Year         178 non-null    int64  
 9   value        178 non-null    float64
 10  units        178 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 15.4+ KB
None


,indicator,category_1,category_2,units,size
0,Industrial Production,Coal,NaN,100 million tons,2
1,Industrial Production,Crude Oil,NaN,10 000 tons,2
2,Industrial Production,Natural Gas,NaN,100 million cu.m,2
3,Industrial Production,Salt,NaN,10 000 tons,2
4,Industrial Production,Refined Edible Vegetable Oil,NaN,10 000 tons,2
...,...,...,...,...,...
84,Industrial Production,Cameras,Digital Cameras,10 000 sets,2
85,Industrial Production,Xerox and Hectograph Printing Equipment(10 000...,NaN,10 000 sets,2
86,Industrial Production,Electricity Generation,NaN,100 million kwh,2
87,Industrial Production,Electricity Generation,Thermal Power,100 million kwh,2


In [271]:
dataframe_list['indo'] = norm_indo

### Income & Consumption

In [272]:
filename = file_list[7]
print(filename)
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

Per capita income and consumption expenditure_人均收支


['6-1  全国居民人均收支情况',
 '6-16  居民人均可支配收入和指数',
 '6-6  城镇居民人均收支情况',
 '6-11  农村居民人均收支情况',
 '6-1  Nationwide Per Capita Inco',
 '6-6  Per Capita Income and Cons',
 '6-16  Per Capita Disposable Inc',
 '6-11  Per Capita Income and Con']

In [273]:
tablename = sheets[4]
print(tablename)
pcin = read_sheet(dataset, tablename)

6-1  Nationwide Per Capita Inco


In [274]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = [0]
for row in pcin.index[1:]:
    if str(pcin[pcin.keys()[2]][row]) == 'nan':
        to_del.append(row)
    else:
        pcin.at[row, pcin.keys()[0]] = re.sub("\s\s+", " ", pcin[pcin.keys()[0]][row])[1:]
pcin.drop(axis=0, index=to_del, inplace=True)
pcin.reset_index(inplace=True, drop=True)

In [275]:
pcin.columns = pcin.iloc[0]
pcin.drop(pcin.index[0], inplace=True)
pcin.reset_index(inplace=True, drop=True)

In [276]:
cats = []
for row in pcin.index:
    try:
        int(pcin[pcin.keys()[0]][row][0])
        pcin.at[row, pcin.keys()[0]] = pcin[pcin.keys()[0]][row].split('.')[1]
    except:
        cats.append(pcin[pcin.keys()[0]][row])

In [277]:
cats

['Disposable Income',
 'Disposable Income in Cash',
 'Consumption Expenditure',
 'Consumption Expenditure on Services ',
 'Consumption Expenditure in Cash']

In [278]:
norm_pcin = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])

for col in range(1, 8):
    for row in pcin.index:
        if pcin.iloc[row,0] in cats:
            indi = pcin.iloc[row,0] + ' Per Capita'
            cat1 = 'Nationwide'
            cat2 = 'Total'
        else:
            cat1 = 'Nationwide'
            cat2 = pcin.iloc[row,0]
            
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': indi,
                    'category_1': cat1,
                    'category_2': cat2,
                    'category_3': None,
                    'Region': 'China',
                    'region_type': 'Country',
                    'Year': int(pcin.keys()[col]),
                    'value': pcin.iloc[row, col],
                    'units': 'yuan'}
        norm_pcin = norm_pcin.append(row_dict, ignore_index=True)
norm_pcin['Year'] = pd.to_numeric(norm_pcin['Year'])

In [279]:
check_norm(norm_pcin)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    203 non-null    object 
 1   table_name   203 non-null    object 
 2   indicator    203 non-null    object 
 3   category_1   203 non-null    object 
 4   category_2   203 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       203 non-null    object 
 7   region_type  203 non-null    object 
 8   Year         203 non-null    int64  
 9   value        203 non-null    float64
 10  units        203 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 17.6+ KB
None


,indicator,category_1,category_2,units,size
0,Disposable Income Per Capita,Nationwide,Total,yuan,7
1,Disposable Income Per Capita,Nationwide,Income of Wages and Salaries,yuan,7
2,Disposable Income Per Capita,Nationwide,Net Business Income,yuan,7
3,Disposable Income Per Capita,Nationwide,Net Income from Property,yuan,7
4,Disposable Income Per Capita,Nationwide,Net Income from Transfer,yuan,7
5,Disposable Income in Cash Per Capita,Nationwide,Total,yuan,7
6,Disposable Income in Cash Per Capita,Nationwide,Income of Wages and Salaries,yuan,7
7,Disposable Income in Cash Per Capita,Nationwide,Net Business Income,yuan,7
8,Disposable Income in Cash Per Capita,Nationwide,Net Income from Property,yuan,7
9,Disposable Income in Cash Per Capita,Nationwide,Net Income from Transfer,yuan,7


In [280]:
dataframe_list['pcin'] = norm_pcin

In [281]:
tablename = sheets[5]
print(tablename)
inuh = read_sheet(dataset, tablename)

6-6  Per Capita Income and Cons


In [282]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = [0]
for row in inuh.index[1:]:
    if str(inuh[inuh.keys()[2]][row]) == 'nan':
        to_del.append(row)
    else:
        inuh.at[row, inuh.keys()[0]] = re.sub("\s\s+", " ", inuh[inuh.keys()[0]][row])[1:]
inuh.drop(axis=0, index=to_del, inplace=True)
inuh.reset_index(inplace=True, drop=True)

In [283]:
inuh.columns = inuh.iloc[0]
inuh.drop(inuh.index[0], inplace=True)
inuh.reset_index(inplace=True, drop=True)

In [284]:
cats = []
for row in inuh.index:
    try:
        int(inuh[inuh.keys()[0]][row][0])
        inuh.at[row, inuh.keys()[0]] = inuh[inuh.keys()[0]][row].split('.')[1]
    except:
        cats.append(inuh[inuh.keys()[0]][row])

In [285]:
cats

['Disposable Income',
 'Disposable Income in Cash',
 'Consumption Expenditure',
 'Consumption Expenditure on Services ',
 'Consumption Expenditure in Cash']

In [286]:
norm_inuh = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])

for col in range(1, 8):
    for row in inuh.index:
        if inuh.iloc[row,0] in cats:
            indi = inuh.iloc[row,0] + ' Per Capita'
            cat1 = 'Urban Households'
            cat2 = 'Total'
        else:
            cat1 = 'Urban Households'
            cat2 = inuh.iloc[row,0]
            
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': indi,
                    'category_1': cat1,
                    'category_2': cat2,
                    'category_3': None,
                    'Region': 'China',
                    'region_type': 'Country',
                    'Year': int(inuh.keys()[col]),
                    'value': inuh.iloc[row, col],
                    'units': 'yuan'}
        norm_inuh = norm_inuh.append(row_dict, ignore_index=True)
norm_inuh['Year'] = pd.to_numeric(norm_inuh['Year'])

In [287]:
check_norm(norm_inuh)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    203 non-null    object 
 1   table_name   203 non-null    object 
 2   indicator    203 non-null    object 
 3   category_1   203 non-null    object 
 4   category_2   203 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       203 non-null    object 
 7   region_type  203 non-null    object 
 8   Year         203 non-null    int64  
 9   value        203 non-null    float64
 10  units        203 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 17.6+ KB
None


,indicator,category_1,category_2,units,size
0,Disposable Income Per Capita,Urban Households,Total,yuan,7
1,Disposable Income Per Capita,Urban Households,Income of Wages and Salaries,yuan,7
2,Disposable Income Per Capita,Urban Households,Net Business Income,yuan,7
3,Disposable Income Per Capita,Urban Households,Net Income from Property,yuan,7
4,Disposable Income Per Capita,Urban Households,Net Income from Transfer,yuan,7
5,Disposable Income in Cash Per Capita,Urban Households,Total,yuan,7
6,Disposable Income in Cash Per Capita,Urban Households,Income of Wages and Salaries,yuan,7
7,Disposable Income in Cash Per Capita,Urban Households,Net Business Income,yuan,7
8,Disposable Income in Cash Per Capita,Urban Households,Net Income from Property,yuan,7
9,Disposable Income in Cash Per Capita,Urban Households,Net Income from Transfer,yuan,7


In [288]:
dataframe_list['inuh'] = norm_inuh

In [289]:
tablename = sheets[6]
print(tablename)
diic= read_sheet(dataset, tablename)

6-16  Per Capita Disposable Inc


In [290]:
for col in range(1, 7):
    if str(diic.iloc[1,col]) == 'nan':
        diic.at[1,diic.keys()[col]] = diic.iloc[1, col-1]
        diic.at[2,diic.keys()[col]] = diic.iloc[2, col-1]

In [291]:
del_rows = []
for row in range(29, 38):
    del_rows.append(row) 
diic.drop(columns='Unnamed: 7', inplace=True)
diic.drop(index=del_rows, inplace=True)

In [292]:
diic = clean_headers(diic, 4)

In [293]:
diic['Region'] = 'China'

In [294]:
diic.keys()[1:-1]

Index(['Nationwide Per Capita Disposable Income of Households Value  (yuan)',
       'Nationwide Per Capita Disposable Income of Households Index(1978=100)',
       'Per Capita Disposable Income of Urban Households Value  (yuan)',
       'Per Capita Disposable Income of Urban Households Index(1978=100)',
       'Per Capita Disposable Income of Rural Households Value  (yuan)',
       'Per Capita Disposable Income of Rural Households Index(1978=100)'],
      dtype='object', name=0)

In [295]:
cat_dic = {'Nationwide Per Capita Disposable Income of Households Value  (yuan)': ['Disposable Income Per Capita', 'Nationwide', 'Total'],
       'Nationwide Per Capita Disposable Income of Households Index(1978=100)': ['Disposable Income Per Capita', 'Nationwide', 'Total'],
       'Per Capita Disposable Income of Urban Households Value  (yuan)': ['Disposable Income Per Capita', 'Urban Households', 'Total'],
       'Per Capita Disposable Income of Urban Households Index(1978=100)': ['Disposable Income Per Capita', 'Urban Households', 'Total'],
       'Per Capita Disposable Income of Rural Households Value  (yuan)': ['Disposable Income Per Capita', 'Rural Households', 'Total'],
       'Per Capita Disposable Income of Rural Households Index(1978=100)': ['Disposable Income Per Capita', 'Rural Households', 'Total']}

unit_dic = {}
for item in diic.keys()[1:-1]:
    if item.find('yuan') > -1:
        unit_dic[item] = 'yuan'
    else:
        unit_dic[item] = 'Percentage (compared to 1978)'

norm_diic = norm_df(diic, cat_dic, unit_dic, filename, tablename)

In [296]:
check_norm(norm_diic)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    150 non-null    object 
 1   table_name   150 non-null    object 
 2   indicator    150 non-null    object 
 3   category_1   150 non-null    object 
 4   category_2   150 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       150 non-null    object 
 7   region_type  150 non-null    object 
 8   Year         150 non-null    int64  
 9   value        150 non-null    float64
 10  units        150 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 13.0+ KB
None


,indicator,category_1,category_2,units,size
0,Disposable Income Per Capita,Nationwide,Total,yuan,25
1,Disposable Income Per Capita,Nationwide,Total,Percentage (compared to 1978),25
2,Disposable Income Per Capita,Urban Households,Total,yuan,25
3,Disposable Income Per Capita,Urban Households,Total,Percentage (compared to 1978),25
4,Disposable Income Per Capita,Rural Households,Total,yuan,25
5,Disposable Income Per Capita,Rural Households,Total,Percentage (compared to 1978),25


In [297]:
dataframe_list['diic'] = norm_diic

In [298]:
tablename = sheets[7]
print(tablename)
inrh = read_sheet(dataset, tablename)

6-11  Per Capita Income and Con


In [299]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = [0]
for row in inrh.index[1:]:
    if str(inrh[inrh.keys()[2]][row]) == 'nan':
        to_del.append(row)
    else:
        label = re.sub("\s\s+", " ", inrh[inrh.keys()[0]][row])
        if label[0] == ' ':
            inrh.at[row, inrh.keys()[0]] = label[1:]
        else:
            inrh.at[row, inrh.keys()[0]] = label
inrh.drop(axis=0, index=to_del, inplace=True)
inrh.reset_index(inplace=True, drop=True)

In [300]:
inrh.columns = inrh.iloc[0]
inrh.drop(inrh.index[0], inplace=True)
inrh.reset_index(inplace=True, drop=True)

In [301]:
cats = []
for row in inrh.index:
    try:
        int(inrh[inrh.keys()[0]][row][0])
        inrh.at[row, inrh.keys()[0]] = inrh[inrh.keys()[0]][row].split('.')[1]
    except:
        cats.append(inrh[inrh.keys()[0]][row])

In [302]:
cats

['Disposable Income',
 'Disposable Income in Cash',
 'Consumption Expenditure',
 'Consumption Expenditure on Services ',
 'Consumption Expenditure in Cash']

In [303]:
norm_inrh = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])

for col in range(1, 8):
    for row in inrh.index:
        if inrh.iloc[row,0] in cats:
            indi = inrh.iloc[row,0] + ' Per Capita'
            cat1 = 'Rural Households'
            cat2 = 'Total'
        else:
            cat1 = 'Rural Households'
            cat2 = inrh.iloc[row,0]
            
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': indi,
                    'category_1': cat1,
                    'category_2': cat2,
                    'category_3': None,
                    'Region': 'China',
                    'region_type': 'Country',
                    'Year': int(inrh.keys()[col]),
                    'value': inrh.iloc[row, col],
                    'units': 'yuan'}
        norm_inrh = norm_inrh.append(row_dict, ignore_index=True)
norm_inrh['Year'] = pd.to_numeric(norm_inrh['Year'])

In [304]:
check_norm(norm_inrh)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    203 non-null    object 
 1   table_name   203 non-null    object 
 2   indicator    203 non-null    object 
 3   category_1   203 non-null    object 
 4   category_2   203 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       203 non-null    object 
 7   region_type  203 non-null    object 
 8   Year         203 non-null    int64  
 9   value        203 non-null    float64
 10  units        203 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 17.6+ KB
None


,indicator,category_1,category_2,units,size
0,Disposable Income Per Capita,Rural Households,Total,yuan,7
1,Disposable Income Per Capita,Rural Households,Income of Wages and Salaries,yuan,7
2,Disposable Income Per Capita,Rural Households,Net Business Income,yuan,7
3,Disposable Income Per Capita,Rural Households,Net Income from Property,yuan,7
4,Disposable Income Per Capita,Rural Households,Net Income from Transfer,yuan,7
5,Disposable Income in Cash Per Capita,Rural Households,Total,yuan,7
6,Disposable Income in Cash Per Capita,Rural Households,Income of Wages and Salaries,yuan,7
7,Disposable Income in Cash Per Capita,Rural Households,Net Business Income,yuan,7
8,Disposable Income in Cash Per Capita,Rural Households,Net Income from Property,yuan,7
9,Disposable Income in Cash Per Capita,Rural Households,Net Income from Transfer,yuan,7


In [305]:
dataframe_list['inrh'] = norm_inrh

### Population

In [306]:
filename = file_list[8]
print(filename)
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

Population_人口


['2-1  Population and Its Composi',
 '2-2  Birth Rate, Death Rate and',
 '2-5  Age Composition and Depend',
 '2-1  人口数及构成',
 '2-2  人口出生率、死亡率和自然增长率',
 '2-5  人口年龄结构',
 '1-1  分地区年末人口数',
 '1-3人口年龄结构和抚养比']

In [307]:
tablename = sheets[0]
print(tablename)
popu = read_sheet(dataset, tablename)

2-1  Population and Its Composi


In [308]:
for col in range(1, 10):
    for row in range(1, 3):
        if str(popu.iloc[row,col]) == 'nan':
            popu.at[row,popu.keys()[col]] = popu.iloc[row, col-1]

In [309]:
popu = clean_headers(popu, 4)

In [310]:
popu.drop(index=[56, 57, 58, 59], inplace=True)

In [311]:
popu.rename(columns={'Total Population (year-end)': 'Total Population',
                    '(10 000 persons) By Residence Rural Proportion': 'By Residence Rural Proportion'},
           inplace=True)

In [312]:
popu['Region'] = 'China'

In [313]:
popu.keys()[1:-1]

Index(['Total Population', 'By Gender Male Population',
       'By Gender Male Proportion', 'By Gender Female Population',
       'By Gender Female Proportion', 'By Residence Urban Population',
       'By Residence Urban Proportion', 'By Residence Rural Population',
       'By Residence Rural Proportion'],
      dtype='object', name=0)

In [314]:
cat_dic = {}
unit_dic = {}
for col in popu.keys()[1:-1]:
    if col == 'Total Population':
        cat_dic[col] = ['Population', 'Total']
        unit_dic[col] = '10 000 persons'
    else:
        cat_dic[col] = ['Population', f"By {col.split(' ')[1]}", col.split(' ')[2]]
        unit_dic[col] = "10 000 persons" if col.split(' ')[3] == 'Population' else "Percentage"

norm_popu = norm_df(popu, cat_dic, unit_dic, filename, tablename)

In [315]:
check_norm(norm_popu)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    504 non-null    object 
 1   table_name   504 non-null    object 
 2   indicator    504 non-null    object 
 3   category_1   504 non-null    object 
 4   category_2   448 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       504 non-null    object 
 7   region_type  504 non-null    object 
 8   Year         504 non-null    int64  
 9   value        504 non-null    float64
 10  units        504 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 43.4+ KB
None


,indicator,category_1,category_2,units,size
0,Population,Total,NaN,10 000 persons,56
1,Population,By Gender,Male,10 000 persons,56
2,Population,By Gender,Male,Percentage,56
3,Population,By Gender,Female,10 000 persons,56
4,Population,By Gender,Female,Percentage,56
5,Population,By Residence,Urban,10 000 persons,56
6,Population,By Residence,Urban,Percentage,56
7,Population,By Residence,Rural,10 000 persons,56
8,Population,By Residence,Rural,Percentage,56


In [316]:
dataframe_list['popu'] = norm_popu

In [317]:
tablename = sheets[1]
print(tablename)
rate = read_sheet(dataset, tablename)

2-2  Birth Rate, Death Rate and


In [318]:
rate = clean_headers(rate, 2)

In [319]:
rate['Region'] = 'China'

In [320]:
rate.rename(columns= {'(‰) Natural Growth Rate': 'Natural Growth Rate'}, inplace=True)
cat_dic = {}
unit_dic = {}
for col in rate.keys()[1:-1]:
    cat_dic[col] = [col]
    unit_dic[col] = '‰'
norm_rate = norm_df(rate, cat_dic, unit_dic, filename, tablename)

In [321]:
check_norm(norm_rate)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    123 non-null    object 
 1   table_name   123 non-null    object 
 2   indicator    123 non-null    object 
 3   category_1   0 non-null      object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       123 non-null    object 
 7   region_type  123 non-null    object 
 8   Year         123 non-null    int64  
 9   value        123 non-null    float64
 10  units        123 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 10.7+ KB
None


,indicator,category_1,units,size
0,Birth Rate,NaN,‰,41
1,Death Rate,NaN,‰,41
2,Natural Growth Rate,NaN,‰,41


In [322]:
dataframe_list['rate'] = norm_rate

In [323]:
tablename = sheets[2]
print(tablename)
depe = read_sheet(dataset, tablename)

2-5  Age Composition and Depend


In [324]:
for col in range(2, 11):
    for row in range(1, 3):
        if str(depe.iloc[row,col]) == 'nan':
            depe.at[row,depe.keys()[col]] = depe.iloc[row, col-1]

In [325]:
for col in depe.keys():
        depe.at[0, col] = (';').join(x for x in list(depe[col][0:4]) if str(x) != 'nan')
depe.columns = depe.iloc[0]
depe.drop(depe.index[0:4], inplace=True)
depe.reset_index(inplace=True, drop=True)

In [326]:
depe.drop(columns='Total;Population;(year-end)', inplace=True)
depe.rename(columns={'(10 000 persons);Elderly;Dependency;Ratio(%)': 'Elderly;Dependency;Ratio(%)'}, inplace=True)
depe['Region'] = 'China'

In [327]:
cat_dic = {}
unit_dic = {}
for col in depe.keys()[1:-1]:
    if col.find('By Age') > -1:
        cat_dic[col] = ['Dependency Rate', 'By Age', col.split(';')[1]]
        unit_dic[col] = "10 000 persons" if col.split(';')[2] == 'Population' else "Percentage"
    else:
        cat_dic[col] = ['Dependency Ratio', col.split(';')[0]]
        unit_dic[col] = 'Percentage'

norm_depe = norm_df(depe, cat_dic, unit_dic, filename, tablename)

In [328]:
check_norm(norm_depe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    288 non-null    object 
 1   table_name   288 non-null    object 
 2   indicator    288 non-null    object 
 3   category_1   288 non-null    object 
 4   category_2   192 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       288 non-null    object 
 7   region_type  288 non-null    object 
 8   Year         288 non-null    int64  
 9   value        288 non-null    float64
 10  units        288 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 24.9+ KB
None


,indicator,category_1,category_2,units,size
0,Dependency Rate,By Age,Aged 0-14,10 000 persons,32
1,Dependency Rate,By Age,Aged 0-14,Percentage,32
2,Dependency Rate,By Age,Aged 15-64,10 000 persons,32
3,Dependency Rate,By Age,Aged 15-64,Percentage,32
4,Dependency Rate,By Age,Aged 65 and Over,10 000 persons,32
5,Dependency Rate,By Age,Aged 65 and Over,Percentage,32
6,Dependency Ratio,Gross,NaN,Percentage,32
7,Dependency Ratio,Children,NaN,Percentage,32
8,Dependency Ratio,Elderly,NaN,Percentage,32


In [329]:
dataframe_list['depe'] = norm_depe

In [330]:
tablename = sheets[6]
print(tablename)
popr = read_sheet(dataset, tablename)

1-1  分地区年末人口数


In [331]:
popr.dropna(axis=0, inplace=True)
popr.drop(index=[3], inplace=True)
popr.drop(columns=popr.keys()[0], inplace=True)
popr.reset_index(inplace=True, drop=True)

In [332]:
popr.columns = popr.iloc[0]
popr.drop(popr.index[0], inplace=True)
popr.reset_index(inplace=True, drop=True)

In [333]:
norm_popr = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])

for col in range(1, len(popr.keys())):
    for row in popr.index:
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': 'Population',
                    'category_1': 'Total',
                    'category_2': None,
                    'category_3': None,
                    'Region': popr.iloc[row, 0],
                    'region_type': 'Province',
                    'Year': int(popr.keys()[col]),
                    'value': popr.iloc[row, col],
                    'units': '10 000 persons'}
        norm_popr = norm_popr.append(row_dict, ignore_index=True)
norm_popr['Year'] = pd.to_numeric(norm_popr['Year'])

In [334]:
check_norm(norm_popr)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    870 non-null    object 
 1   table_name   870 non-null    object 
 2   indicator    870 non-null    object 
 3   category_1   870 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       870 non-null    object 
 7   region_type  870 non-null    object 
 8   Year         870 non-null    int64  
 9   value        870 non-null    float64
 10  units        870 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 74.9+ KB
None


,indicator,category_1,units,size
0,Population,Total,10 000 persons,870


In [335]:
dataframe_list['popr'] = norm_popr

### Transportation

In [336]:
filename = file_list[9]
print(filename)
dataset = pd.ExcelFile(f'{filename}.xlsx')
sheets = dataset.sheet_names
sheets

Transportation_交通


['16-2  交通运输业基本情况',
 '16-3  运 输 线 路 长 度',
 '16-6  客   运   量',
 '16-7  旅  客  周  转  量',
 '16-8  货   运   量',
 '16-9  货  物  周  转  量',
 '16-10  旅 客 运 输 平 均 运 距',
 '16-11  货 物 运 输 平 均 运 距',
 '16-19  高速铁路基本情况',
 '16-20  民 用 汽 车 拥 有 量',
 '16-21  私 人 汽 车 拥 有 量',
 '16-30  民用航空运输量及通用航空飞行时间',
 '16-2  Basic Conditions of Trans',
 '16-3  Length of Transport Route',
 '16-6  Passenger Traffic',
 '16-7  Passenger-Kilometers',
 '16-8  Freight Traffic',
 '16-9  Freight Ton-Kilometers',
 '16-10  Average Transport Distan',
 '16-11  Average Transport Distan',
 '16-19  Basic Statistics of High',
 '16-20  Possession of Civil Vehi',
 '16-21  Possession of Private Ve',
 '16-30  Civil Aviation Traffic a']

In [337]:
tablename = sheets[13]
print(tablename)
rout = read_sheet(dataset, tablename)

16-3  Length of Transport Route


In [338]:
rout = clean_headers(rout, 6)

In [339]:
rout.drop(index=[33, 34, 35, 36, 37, 38, 39], inplace=True)

In [340]:
rout['Region'] = 'China'

In [341]:
rout.keys()

Index(['Year', 'Length of Railways in Operation', 'Electrified Railways',
       'Length of Highways', 'Expressway',
       'Length of Navigable Inland Waterways',
       'Length of Regular Civil Aviation Routes', 'International Routes',
       '(10 000 km) Length of Petroleum and Gas Pipelines', 'Region'],
      dtype='object', name=0)

In [342]:
cat_dic = {'Length of Railways in Operation': ['Length of Railways in Operation', 'Total'],
           'Electrified Railways': ['Length of Railways in Operation', 'Electrified Railways'],
           'Length of Highways': ['Length of Highways', 'Total'],
           'Expressway': ['Length of Highways', 'Expressway'],
       'Length of Navigable Inland Waterways': ['Length of Navigable Inland Waterways'],
       'Length of Regular Civil Aviation Routes': ['Length of Regular Civil Aviation Routes', 'Total'],
           'International Routes': ['Length of Regular Civil Aviation Routes', 'International Routes'],
       '(10 000 km) Length of Petroleum and Gas Pipelines': ['Length of Petroleum and Gas Pipelines']}
unit_dic = {}
for col in rout.keys()[1:-1]:
    unit_dic[col] = '10 000 km'
norm_rout = norm_df(rout, cat_dic, unit_dic, filename, tablename)

In [343]:
check_norm(norm_rout)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    264 non-null    object 
 1   table_name   264 non-null    object 
 2   indicator    264 non-null    object 
 3   category_1   198 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       264 non-null    object 
 7   region_type  264 non-null    object 
 8   Year         264 non-null    int64  
 9   value        261 non-null    float64
 10  units        264 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 22.8+ KB
None


,indicator,category_1,units,size
0,Length of Railways in Operation,Total,10 000 km,33
1,Length of Railways in Operation,Electrified Railways,10 000 km,33
2,Length of Highways,Total,10 000 km,33
3,Length of Highways,Expressway,10 000 km,33
4,Length of Navigable Inland Waterways,NaN,10 000 km,33
5,Length of Regular Civil Aviation Routes,Total,10 000 km,33
6,Length of Regular Civil Aviation Routes,International Routes,10 000 km,33
7,Length of Petroleum and Gas Pipelines,NaN,10 000 km,33


In [344]:
dataframe_list['rout'] = norm_rout

In [345]:
tablename = sheets[14]
print(tablename)
ptraf = read_sheet(dataset, tablename)

16-6  Passenger Traffic


In [346]:
ptraf.drop(index=[0, 36, 37, 38, 39, 40, 41, 42, 43, 44], columns=ptraf.keys()[6:], inplace=True)
ptraf.reset_index(inplace=True, drop=True)

In [347]:
ptraf = clean_headers(ptraf, 2)

In [348]:
ptraf['Region'] = 'China'

In [349]:
cat_dic = {}
unit_dic = {}
for col in ptraf.keys()[1:-1]:
    cat_dic[col] = ['Passenger Traffic', col]
    unit_dic[col] = '10 000 persons'
norm_ptraf = norm_df(ptraf, cat_dic, unit_dic, filename, tablename)

In [350]:
check_norm(norm_ptraf)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    165 non-null    object 
 1   table_name   165 non-null    object 
 2   indicator    165 non-null    object 
 3   category_1   165 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       165 non-null    object 
 7   region_type  165 non-null    object 
 8   Year         165 non-null    int64  
 9   value        165 non-null    float64
 10  units        165 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 14.3+ KB
None


,indicator,category_1,units,size
0,Passenger Traffic,Total,10 000 persons,33
1,Passenger Traffic,Railways,10 000 persons,33
2,Passenger Traffic,Highways,10 000 persons,33
3,Passenger Traffic,Waterways,10 000 persons,33
4,Passenger Traffic,Civil Aviation,10 000 persons,33


In [351]:
dataframe_list['ptraf'] = norm_ptraf

In [352]:
tablename = sheets[15]
print(tablename)
pkil = read_sheet(dataset, tablename)

16-7  Passenger-Kilometers


In [353]:
pkil.drop(index=[0], inplace=True)
pkil.reset_index(inplace=True, drop=True)

In [354]:
pkil = clean_headers(pkil, 2)

In [355]:
pkil['Region'] = 'China'

In [356]:
cat_dic = {}
unit_dic = {}
for col in pkil.keys()[1:-1]:
    cat_dic[col] = ['Passenger Traffic per kilometer', col]
    unit_dic[col] = 'passengers per kilometer'
norm_pkil = norm_df(pkil, cat_dic, unit_dic, filename, tablename)

In [357]:
check_norm(norm_pkil)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    165 non-null    object 
 1   table_name   165 non-null    object 
 2   indicator    165 non-null    object 
 3   category_1   165 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       165 non-null    object 
 7   region_type  165 non-null    object 
 8   Year         165 non-null    int64  
 9   value        165 non-null    float64
 10  units        165 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 14.3+ KB
None


,indicator,category_1,units,size
0,Passenger Traffic per kilometer,Total,passengers per kilometer,33
1,Passenger Traffic per kilometer,Railways,passengers per kilometer,33
2,Passenger Traffic per kilometer,Highways,passengers per kilometer,33
3,Passenger Traffic per kilometer,Waterways,passengers per kilometer,33
4,Passenger Traffic per kilometer,Civil Aviation,passengers per kilometer,33


In [358]:
dataframe_list['pkil'] = norm_pkil

In [359]:
tablename = sheets[16]
print(tablename)
fret = read_sheet(dataset, tablename)

16-8  Freight Traffic


In [360]:
fret.drop(index=[0, 37, 38], inplace=True)
fret.reset_index(inplace=True, drop=True)

In [361]:
fret = clean_headers(fret, 3)

In [362]:
fret['Region'] = 'China'

In [363]:
cat_dic = {}
unit_dic = {}
for col in fret.keys()[1:-1]:
    if col == 'Waterways':
        cat_dic[col] = ['Freight Traffic', col, 'Total']
    elif col == 'Ocean':
        cat_dic[col] = ['Freight Traffic', 'Waterways', col]
    else:
        cat_dic[col] = ['Freight Traffic', col]
    unit_dic[col] = '10 000 tons'
norm_fret = norm_df(fret, cat_dic, unit_dic, filename, tablename)

In [364]:
check_norm(norm_fret)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    231 non-null    object 
 1   table_name   231 non-null    object 
 2   indicator    231 non-null    object 
 3   category_1   231 non-null    object 
 4   category_2   66 non-null     object 
 5   category_3   0 non-null      object 
 6   Region       231 non-null    object 
 7   region_type  231 non-null    object 
 8   Year         231 non-null    int64  
 9   value        231 non-null    float64
 10  units        231 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 20.0+ KB
None


,indicator,category_1,category_2,units,size
0,Freight Traffic,Total,NaN,10 000 tons,33
1,Freight Traffic,Railways,NaN,10 000 tons,33
2,Freight Traffic,Highways,NaN,10 000 tons,33
3,Freight Traffic,Waterways,Total,10 000 tons,33
4,Freight Traffic,Waterways,Ocean,10 000 tons,33
5,Freight Traffic,Civil Aviation,NaN,10 000 tons,33
6,Freight Traffic,Petroleum and Gas Pipelines,NaN,10 000 tons,33


In [365]:
dataframe_list['fret'] = norm_fret

In [366]:
tablename = sheets[17]
print(tablename)
fretk = read_sheet(dataset, tablename)

16-9  Freight Ton-Kilometers


In [367]:
fretk.drop(index=[0], inplace=True)
fretk.reset_index(inplace=True, drop=True)

In [368]:
frekt = clean_headers(fretk, 3)

In [369]:
fretk['Region'] = 'China'

In [370]:
cat_dic = {}
unit_dic = {}
for col in fretk.keys()[1:-1]:
    if col == 'Waterways':
        cat_dic[col] = ['Freight Traffic', col, 'Total']
    elif col == 'Ocean':
        cat_dic[col] = ['Freight Traffic', 'Waterways', col]
    else:
        cat_dic[col] = ['Freight Traffic', col]
    unit_dic[col] = '100 million tons per km'
norm_fretk = norm_df(fretk, cat_dic, unit_dic, filename, tablename)

In [371]:
check_norm(norm_fretk)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    231 non-null    object 
 1   table_name   231 non-null    object 
 2   indicator    231 non-null    object 
 3   category_1   231 non-null    object 
 4   category_2   66 non-null     object 
 5   category_3   0 non-null      object 
 6   Region       231 non-null    object 
 7   region_type  231 non-null    object 
 8   Year         231 non-null    int64  
 9   value        231 non-null    float64
 10  units        231 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 20.0+ KB
None


,indicator,category_1,category_2,units,size
0,Freight Traffic,Total,NaN,100 million tons per km,33
1,Freight Traffic,Railways,NaN,100 million tons per km,33
2,Freight Traffic,Highways,NaN,100 million tons per km,33
3,Freight Traffic,Waterways,Total,100 million tons per km,33
4,Freight Traffic,Waterways,Ocean,100 million tons per km,33
5,Freight Traffic,Civil Aviation,NaN,100 million tons per km,33
6,Freight Traffic,Petroleum and Gas Pipelines,NaN,100 million tons per km,33


In [372]:
dataframe_list['fretk'] = norm_fretk

In [373]:
tablename = sheets[18]
print(tablename)
ptrav = read_sheet(dataset, tablename)

16-10  Average Transport Distan


In [374]:
ptrav.drop(index=[0], inplace=True)
ptrav.reset_index(inplace=True, drop=True)

In [375]:
ptrav = clean_headers(ptrav, 1)

In [376]:
ptrav['Region'] = 'China'

In [377]:
cat_dic = {}
unit_dic = {}
for col in ptrav.keys()[1:-1]:
    cat_dic[col] = ['Travel Distance per passenger', col]
    unit_dic[col] = 'km'
norm_ptrav = norm_df(ptrav, cat_dic, unit_dic, filename, tablename)

In [378]:
check_norm(norm_ptrav)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    165 non-null    object 
 1   table_name   165 non-null    object 
 2   indicator    165 non-null    object 
 3   category_1   165 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       165 non-null    object 
 7   region_type  165 non-null    object 
 8   Year         165 non-null    int64  
 9   value        165 non-null    float64
 10  units        165 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 14.3+ KB
None


,indicator,category_1,units,size
0,Travel Distance per passenger,Total,km,33
1,Travel Distance per passenger,Railways,km,33
2,Travel Distance per passenger,Highways,km,33
3,Travel Distance per passenger,Waterways,km,33
4,Travel Distance per passenger,Civil Aviation,km,33


In [379]:
dataframe_list['ptrav'] = norm_ptrav

In [380]:
tablename = sheets[19]
print(tablename)
fdis = read_sheet(dataset, tablename)

16-11  Average Transport Distan


In [381]:
fdis.drop(index=[0], inplace=True)
fdis.reset_index(inplace=True, drop=True)

In [382]:
fdis = clean_headers(fdis, 2)

In [383]:
fdis['Region'] = 'China'

In [384]:
cat_dic = {}
unit_dic = {}
for col in fdis.keys()[1:-1]:
    cat_dic[col] = ['Average Freight Transport Distance', col]
    unit_dic[col] = 'km'
norm_fdis = norm_df(fdis, cat_dic, unit_dic, filename, tablename)

In [385]:
check_norm(norm_fdis)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    198 non-null    object 
 1   table_name   198 non-null    object 
 2   indicator    198 non-null    object 
 3   category_1   198 non-null    object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       198 non-null    object 
 7   region_type  198 non-null    object 
 8   Year         198 non-null    int64  
 9   value        198 non-null    float64
 10  units        198 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 17.1+ KB
None


,indicator,category_1,units,size
0,Average Freight Transport Distance,Total,km,33
1,Average Freight Transport Distance,Railways,km,33
2,Average Freight Transport Distance,Highways,km,33
3,Average Freight Transport Distance,Waterways,km,33
4,Average Freight Transport Distance,Civil Aviation,km,33
5,Average Freight Transport Distance,Petroleum and Gas Pipelines,km,33


In [386]:
dataframe_list['fdis'] = norm_fdis

In [387]:
tablename = sheets[20]
print(tablename)
hsp = read_sheet(dataset, tablename)

16-19  Basic Statistics of High


In [388]:
hsp = clean_headers(hsp, 5)

In [389]:
hsp['Region'] = 'China'

In [390]:
cat_dic = {'Length in Operation (km)': ['Highspeed Railway in Operation'],
       'Percentage of Length of Railways in Operation (%)': ['Highspeed Railway in Operation'],
       'Passenger Traffic (10 000 persons)': ['Highspeed Passenger Traffic'],
       'Percentage of Railway Passenger Traffic (%)': ['Highspeed Passenger Traffic'],
       'Passenger- Kilometers (100 million passenger-km)': ['Highspeed Passenger Traffic'],
       'Percentage of Railway Passenger-Kilometers (%)': ['Highspeed Passenger Traffic']}
unit_dic = {'Length in Operation (km)': 'Kilometers',
       'Percentage of Length of Railways in Operation (%)': 'Percentage',
       'Passenger Traffic (10 000 persons)': '10 000 persons',
       'Percentage of Railway Passenger Traffic (%)': 'Percentage',
       'Passenger- Kilometers (100 million passenger-km)': '100 million passenger/km',
       'Percentage of Railway Passenger-Kilometers (%)': 'Percentage'}
norm_hsp = norm_df(hsp, cat_dic, unit_dic, filename, tablename)

In [391]:
check_norm(norm_hsp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    72 non-null     object 
 1   table_name   72 non-null     object 
 2   indicator    72 non-null     object 
 3   category_1   0 non-null      object 
 4   category_2   0 non-null      object 
 5   category_3   0 non-null      object 
 6   Region       72 non-null     object 
 7   region_type  72 non-null     object 
 8   Year         72 non-null     int64  
 9   value        72 non-null     float64
 10  units        72 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 6.3+ KB
None


,indicator,category_1,units,size
0,Highspeed Railway in Operation,NaN,Kilometers,12
1,Highspeed Railway in Operation,NaN,Percentage,12
2,Highspeed Passenger Traffic,NaN,10 000 persons,12
3,Highspeed Passenger Traffic,NaN,Percentage,24
4,Highspeed Passenger Traffic,NaN,100 million passenger/km,12


In [392]:
dataframe_list['hsp'] = norm_hsp

In [393]:
tablename = sheets[21]
print(tablename)
cveh = read_sheet(dataset, tablename)

16-20  Possession of Civil Vehi


In [394]:
cveh.drop(index=[0, 56, 57, 58], inplace=True)
cveh.reset_index(inplace=True, drop=True)

In [395]:
cveh.at[1, 'Unnamed: 4'] = 'Medium-v'

In [396]:
cveh = clean_headers(cveh, 3)

In [397]:
cveh = year_region(cveh, yr_col = cveh.keys()[0])

In [398]:
ren_cols = {}
for col in cveh.keys()[1:-1]:
    if col.find(' (') > -1:
        ren_cols[col] = col.split(' (')[0]
cveh.rename(columns=ren_cols, inplace=True)

In [399]:
cveh.keys()[1:-1]

Index(['Total', 'Passenger Vehicles', 'Large', 'Medium-v', 'Small', 'Minicar',
       'Trucks', 'Heavy', 'Medium', 'Light', 'Mini', 'Others',
       'Number of Motor Drivers', 'Automobile Drivers'],
      dtype='object', name=0)

In [400]:
cat_dic = {}
unit_dic = {}
for num in range(1, len(cveh.keys())-1):
    col = cveh.keys()[num]
    if num in [1, 12]:
        cat1 = col
        cat2 = None
    elif num in [2, 7]:
        cat1 = col
        cat2 = 'Total'
    elif num in [3, 5, 6, 8, 9, 10, 11, 14]:
        cat2 = col
    elif num == 4:
        cat2 = 'Medium'
    elif num == 13:
        cat1 = 'Motor Drivers'
        cat2 = 'Total'
    
    if num in [13, 14]:
        units = '10 000 persons'
    else:
        units = '10 000 units'
        
    cat_dic[col] = ['Civil Vehicles', cat1, cat2]
    unit_dic[col] = units
norm_cveh = norm_df(cveh, cat_dic, unit_dic, filename, tablename)

In [401]:
check_norm(norm_cveh)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    728 non-null    object 
 1   table_name   728 non-null    object 
 2   indicator    728 non-null    object 
 3   category_1   728 non-null    object 
 4   category_2   624 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       728 non-null    object 
 7   region_type  728 non-null    object 
 8   Year         728 non-null    int64  
 9   value        670 non-null    float64
 10  units        728 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 62.7+ KB
None


,indicator,category_1,category_2,units,size
0,Civil Vehicles,Total,NaN,10 000 units,52
1,Civil Vehicles,Passenger Vehicles,Total,10 000 units,52
2,Civil Vehicles,Passenger Vehicles,Large,10 000 units,52
3,Civil Vehicles,Passenger Vehicles,Medium,10 000 units,52
4,Civil Vehicles,Passenger Vehicles,Small,10 000 units,52
5,Civil Vehicles,Passenger Vehicles,Minicar,10 000 units,52
6,Civil Vehicles,Trucks,Total,10 000 units,52
7,Civil Vehicles,Trucks,Heavy,10 000 units,52
8,Civil Vehicles,Trucks,Medium,10 000 units,52
9,Civil Vehicles,Trucks,Light,10 000 units,52


In [402]:
dataframe_list['cveh'] = norm_cveh

In [403]:
tablename = sheets[22]
print(tablename)
pveh = read_sheet(dataset, tablename)

16-21  Possession of Private Ve


In [404]:
pveh.drop(index=[0], inplace=True)
pveh.reset_index(inplace=True, drop=True)

In [405]:
pveh.at[2, 'Unnamed: 4'] = 'Medium-v'

In [406]:
pveh = clean_headers(pveh, 3)

In [407]:
pveh = year_region(pveh, yr_col = pveh.keys()[0])

In [408]:
cveh.keys()[1:-1]

Index(['Total', 'Passenger Vehicles', 'Large', 'Medium-v', 'Small', 'Minicar',
       'Trucks', 'Heavy', 'Medium', 'Light', 'Mini', 'Others',
       'Number of Motor Drivers', 'Automobile Drivers'],
      dtype='object', name=0)

In [409]:
pveh.keys()[1:-1]

Index(['Total', 'Passenger Vehicles', 'Large', 'Medium-v', 'Small', 'Minicar',
       'Trucks', 'Heavy', 'Medium', 'Light', 'Mini', 'Others'],
      dtype='object', name=0)

In [410]:
cat_dic = {}
unit_dic = {}
for num in range(1, len(pveh.keys())-1):
    col = cveh.keys()[num]
    if num in [1, 12]:
        cat1 = col
        cat2 = None
    elif num in [2, 7]:
        cat1 = col
        cat2 = 'Total'
    elif num in [3, 5, 6, 8, 9, 10, 11, 14]:
        cat2 = col
    elif num == 4:
        cat2 = 'Medium'
        
    cat_dic[col] = ['Private Vehicles', cat1, cat2]
    unit_dic[col] = '10 000 units'
norm_pveh = norm_df(pveh, cat_dic, unit_dic, filename, tablename)

In [411]:
check_norm(norm_pveh)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    600 non-null    object 
 1   table_name   600 non-null    object 
 2   indicator    600 non-null    object 
 3   category_1   600 non-null    object 
 4   category_2   500 non-null    object 
 5   category_3   0 non-null      object 
 6   Region       600 non-null    object 
 7   region_type  600 non-null    object 
 8   Year         600 non-null    int64  
 9   value        563 non-null    float64
 10  units        600 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 51.7+ KB
None


,indicator,category_1,category_2,units,size
0,Private Vehicles,Total,NaN,10 000 units,50
1,Private Vehicles,Passenger Vehicles,Total,10 000 units,50
2,Private Vehicles,Passenger Vehicles,Large,10 000 units,50
3,Private Vehicles,Passenger Vehicles,Medium,10 000 units,50
4,Private Vehicles,Passenger Vehicles,Small,10 000 units,50
5,Private Vehicles,Passenger Vehicles,Minicar,10 000 units,50
6,Private Vehicles,Trucks,Total,10 000 units,50
7,Private Vehicles,Trucks,Heavy,10 000 units,50
8,Private Vehicles,Trucks,Medium,10 000 units,50
9,Private Vehicles,Trucks,Light,10 000 units,50


In [412]:
dataframe_list['pveh'] = norm_pveh

In [413]:
tablename = sheets[23]
print(tablename)
avtr = read_sheet(dataset, tablename)

16-30  Civil Aviation Traffic a


In [414]:
#remove empty rows, add the text to the previous row (indicator names split into several rows)
to_del = []
for row in avtr.index:
    if str(avtr[avtr.keys()[2]][row]) == 'nan':
        avtr.iloc[row-1, 0] = avtr[avtr.keys()[0]][row-1] + avtr[avtr.keys()[0]][row]
        to_del.append(row)
avtr.drop(axis=0, index=to_del, inplace=True)
avtr.reset_index(inplace=True, drop=True)

In [415]:
avtr.columns = avtr.iloc[0]
avtr.drop(avtr.index[0], inplace=True)
avtr.reset_index(inplace=True, drop=True)

In [416]:
cat_dic = {0: ['Air Passenger Traffic', 'Total'],
 1: ['Air Passenger Traffic', 'International Routes'],
 2: ['Air Passenger Traffic', 'Domestic Routes', 'Total'],
 3: ['Air Passenger Traffic', 'Domestic Routes', 'Routes of Hong Kong, Macao and Taiwan'],
 4: ['Air Passenger Traffic', 'Total'],
 5: ['Air Passenger Traffic', 'International Routes'],
 6: ['Air Passenger Traffic', 'Domestic Routes', 'Total'],
 7: ['Air Passenger Traffic', 'Domestic Routes', 'Routes of Hong Kong, Macao and Taiwan'],
 8: ['Air Freight Traffic', 'Total'],
 9: ['Air Freight Traffic', 'International Routes'],
 10: ['Air Freight Traffic', 'Domestic Routes', 'Total'],
 11: ['Air Freight Traffic', 'Domestic Routes', 'Routes of Hong Kong, Macao and Taiwan'],
 12: ['Air Freight Traffic', 'Total'],
 13: ['Air Freight Traffic', 'International Routes'],
 14: ['Air Freight Traffic', 'Domestic Routes', 'Total'],
 15: ['Air Freight Traffic', 'Domestic Routes', 'Routes of Hong Kong, Macao and Taiwan'],
 16: ['Total Air Traffic', 'Total'],
 17: ['Total Air Traffic', 'International Routes'],
 18: ['Total Air Traffic', 'Domestic Routes'],
 19: ['Total Air Traffic', 'Routes of Hong Kong, Macao and Taiwan'],
 20: ['Flying Time of General Aviation', 'Total'],
 21: ['Flying Time of General Aviation', 'Flight for Agriculture and Forestry Operation', 'Total'],
 22: ['Flying Time of General Aviation', 'Flight for Agriculture and Forestry Operation', 'Forest Protection Service'],
 23: ['Flying Time of General Aviation', 'Flight for Agriculture and Forestry Operation', 'Afforestation'],
 24: ['Flying Time of General Aviation', 'Flight for Industrial Operation']}
unit_dic = {0: '10 000 persons',
 1: '10 000 persons',
 2: '10 000 persons',
 3: '10 000 persons',
 4: '10 000 persons/km',
 5: '10 000 persons/km',
 6: '10 000 persons/km',
 7: '10 000 persons/km',
 8: 'tons',
 9: 'tons',
 10: 'tons',
 11: 'tons',
 12: '10 000 ton/km',
 13: '10 000 ton/km',
 14: '10 000 ton/km',
 15: '10 000 ton/km',
 16: '10 000 ton/km',
 17: '10 000 ton/km',
 18: '10 000 ton/km',
 19: '10 000 ton/km',
 20: 'hr',
 21: 'hr',
 22: 'hr',
 23: 'hr',
 24: 'hr'}

In [417]:
norm_avtr = pd.DataFrame(columns=['file_name', 'table_name', 'indicator', 'category_1', 'category_2', 'category_3', 'Region',
       'region_type', 'Year', 'value', 'units'])
for col in range(1, 7):
    for row in avtr.index:
        indi = cat_dic[row][0]
        cat1 = cat_dic[row][1]
        try:
            cat2 = cat_dic[row][2]
        except:
            cat2 = None
        
        row_dict = {'file_name': filename,
                    'table_name': tablename,
                    'indicator': indi,
                    'category_1': cat1,
                    'category_2': cat2,
                    'category_3': None,
                    'Region': 'China',
                    'region_type': 'Country',
                    'Year': avtr.keys()[col],
                    'value': avtr.iloc[row, col],
                    'units': unit_dic[row]}
        norm_avtr = norm_avtr.append(row_dict, ignore_index=True)
norm_avtr['Year'] = pd.to_numeric(norm_avtr['Year'])

In [418]:
check_norm(norm_avtr)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   file_name    150 non-null    object 
 1   table_name   150 non-null    object 
 2   indicator    150 non-null    object 
 3   category_1   150 non-null    object 
 4   category_2   66 non-null     object 
 5   category_3   0 non-null      object 
 6   Region       150 non-null    object 
 7   region_type  150 non-null    object 
 8   Year         150 non-null    float64
 9   value        142 non-null    float64
 10  units        150 non-null    object 
dtypes: float64(2), object(9)
memory usage: 13.0+ KB
None


,indicator,category_1,category_2,units,size
0,Air Passenger Traffic,Total,NaN,10 000 persons,6
1,Air Passenger Traffic,International Routes,NaN,10 000 persons,6
2,Air Passenger Traffic,Domestic Routes,Total,10 000 persons,6
3,Air Passenger Traffic,Domestic Routes,"Routes of Hong Kong, Macao and Taiwan",10 000 persons,6
4,Air Passenger Traffic,Total,NaN,10 000 persons/km,6
5,Air Passenger Traffic,International Routes,NaN,10 000 persons/km,6
6,Air Passenger Traffic,Domestic Routes,Total,10 000 persons/km,6
7,Air Passenger Traffic,Domestic Routes,"Routes of Hong Kong, Macao and Taiwan",10 000 persons/km,6
8,Air Freight Traffic,Total,NaN,tons,6
9,Air Freight Traffic,International Routes,NaN,tons,6


In [419]:
dataframe_list['avtr'] = norm_avtr

In [420]:
norm_avtr.tail(10)

,file_name,table_name,indicator,category_1,category_2,category_3,Region,region_type,Year,value,units
140,Transportation_交通,16-30 Civil Aviation Traffic a,Air Freight Traffic,Domestic Routes,"Routes of Hong Kong, Macao and Taiwan",None,China,Country,2019.0,28056.96,10 000 ton/km
141,Transportation_交通,16-30 Civil Aviation Traffic a,Total Air Traffic,Total,None,None,China,Country,2019.0,12932529.99,10 000 ton/km
142,Transportation_交通,16-30 Civil Aviation Traffic a,Total Air Traffic,International Routes,None,None,China,Country,2019.0,4637425.69,10 000 ton/km
143,Transportation_交通,16-30 Civil Aviation Traffic a,Total Air Traffic,Domestic Routes,None,None,China,Country,2019.0,8295104.30,10 000 ton/km
144,Transportation_交通,16-30 Civil Aviation Traffic a,Total Air Traffic,"Routes of Hong Kong, Macao and Taiwan",None,None,China,Country,2019.0,168954.40,10 000 ton/km
145,Transportation_交通,16-30 Civil Aviation Traffic a,Flying Time of General Aviation,Total,None,None,China,Country,2019.0,1065011.00,hr
146,Transportation_交通,16-30 Civil Aviation Traffic a,Flying Time of General Aviation,Flight for Agriculture and Forestry Operation,Total,None,China,Country,2019.0,NaN,hr
147,Transportation_交通,16-30 Civil Aviation Traffic a,Flying Time of General Aviation,Flight for Agriculture and Forestry Operation,Forest Protection Service,None,China,Country,2019.0,NaN,hr
148,Transportation_交通,16-30 Civil Aviation Traffic a,Flying Time of General Aviation,Flight for Agriculture and Forestry Operation,Afforestation,None,China,Country,2019.0,NaN,hr
149,Transportation_交通,16-30 Civil Aviation Traffic a,Flying Time of General Aviation,Flight for Industrial Operation,None,None,China,Country,2019.0,NaN,hr


## Merge all normalised dataframes

In [439]:
final_df.keys()

Index(['file_name', 'table_name', 'indicator', 'category_1', 'category_2',
       'category_3', 'Region', 'region_type', 'Year', 'value', 'units'],
      dtype='object')

In [444]:
soc_df = final_df.copy()
soc_df.drop(columns='table_name', inplace=True)
soc_df['group'] = 'Socioeconomic'
soc_df.rename(columns={'file_name': 'subgroup', 'Year': 'year', 'Region':'region'}, inplace=True)
soc_df['subgroup'] = [x.split('_')[0] for x in soc_df['group']]
soc_df = soc_df[['group', 'subgroup', 'indicator', 'category_1', 'category_2', 'category_3',
       'region', 'region_type', 'year', 'value', 'units']]

In [448]:
soc_df.to_csv('socioecon.csv', index=False)

In [421]:
final_df = pd.concat(dataframe_list.values())
final_df.reset_index(inplace=True, drop=True)

In [437]:
final_df.to_csv('socioecon_clean.csv', index=False)

In [432]:
class_df = final_df.groupby(list(final_df.keys()[:6]), sort=False, as_index=False, dropna=False).size()

In [423]:
for filename in final_df['file_name'].unique():
    print(filename)
    print(final_df[final_df['file_name']==filename]['indicator'].unique())
    print(' ')

Agriculture_农业
['Total Power of Agricultural Machinery' 'Farm Machinery'
 'Irrigated Area of Cultivated Land' 'Consumption of Chemical Fertilizers'
 'Irrigated Areas' 'Effective Irrigated Area' 'Reservoirs'
 'Capacity of Reservoirs' 'Water-saving Irrigated Area '
 'Area with Flood Prevention Measures'
 'Area with Soil Erosion under Control' 'Total Length of Dikes'
 'Area of Land Protected by Dikes']
 
Building_建筑
['Building Floorspace']
 
Employment_就业
['Employment' 'Labour Force' 'Employed persons'
 'Unemployed persons in Urban areas' 'Employed People']
 
Exchange Rate_汇率
['Exchange Rate']
 
GDP_国内生产总值
['Gross National Income' 'Gross Domestic Product' 'GDP'
 'GDP (constant price)' 'Gross National Income (constant price)'
 'Gross Domestic Product (constant price)'
 'GDP per capita (constant price)' 'GNI per capita (constant price)'
 'Gross National Income (compared to 1978)'
 'Gross Domestic Product (compared to 1978)' 'GDP (compared to 1978)'
 'GDP per capita (compared to 1978)' 'GNI 

In [433]:
class_df.head()

,file_name,table_name,indicator,category_1,category_2,category_3,size
0,Agriculture_农业,12-4 Major Agricultural Machin,Total Power of Agricultural Machinery,NaN,NaN,NaN,52
1,Agriculture_农业,12-4 Major Agricultural Machin,Farm Machinery,Large and Medium-sized Tractors Number,NaN,NaN,52
2,Agriculture_农业,12-4 Major Agricultural Machin,Farm Machinery,Towing Farm Machinery,NaN,NaN,52
3,Agriculture_农业,12-4 Major Agricultural Machin,Farm Machinery,Small Tractors Number,NaN,NaN,52
4,Agriculture_农业,12-5 Irrigated Area of Cultiva,Irrigated Area of Cultivated Land,NaN,NaN,NaN,52


## Further cleaning

In [46]:
socioecon = pd.read_csv('../data/Socio-economic Data/socioecon_clean.csv')

### Re-arrange some indicators

In [47]:
socioecon.keys()

Index(['file_name', 'table_name', 'indicator', 'category_1', 'category_2',
       'category_3', 'Region', 'region_type', 'Year', 'value', 'units'],
      dtype='object')

In [48]:
socioecon.replace({'indicator': {'Reservoirs': 'Number of Reservoirs'}, 'category_1': {'Number of Reservoirs  (unit)': 'Total'}}, inplace=True)
socioecon.loc[(socioecon['indicator']=='Capacity of Reservoirs')&(socioecon['category_1']=='Total'), 'units'] = '100m m3'
socioecon.loc[(socioecon['indicator']=='Capacity of Reservoirs')&(socioecon['category_1']=='Total'), 'units'] = '100m m3'

In [49]:
socioecon.drop(socioecon.loc[(socioecon['indicator']=='Employment')&(socioecon['category_1']=='Terciary Industry')].index, inplace=True)
socioecon.loc[(socioecon['indicator']=='Employed persons')&(socioecon['category_1']=='Industry breakdown'), 'indicator'] = 'Employed persons (Industry breakdown)'
socioecon.loc[(socioecon['indicator']=='Employed persons')&(socioecon['category_1']=='Urban/Rural breakdown'), 'indicator'] = 'Employed persons (Urban/Rural breakdown)'
socioecon.loc[(socioecon['file_name']=='Employment_就业')&(socioecon['indicator']=='Employed People'), 'indicator'] = 'Employed People in urban areas'
for item in ['Employed persons (Industry breakdown)', 'Employed persons (Urban/Rural breakdown)', 'Employed People in urban areas']:
    socioecon.loc[socioecon['indicator']==item, 'category_1'] = socioecon.loc[socioecon['indicator']==item]['category_2']
    socioecon.loc[socioecon['indicator']==item, 'category_2'] = socioecon.loc[socioecon['indicator']==item]['category_3']
    socioecon.loc[socioecon['indicator']==item, 'category_3'] = None

In [50]:
summary_socio = socioecon.groupby(['file_name', 'indicator', 'category_1', 'category_2',
       'category_3'], sort=False, as_index=False, dropna=False).size()

In [51]:
for indicator in ['GDP', 'GDP (constant price)', 'GDP (compared to 1978)', 'GDP (main contributors to increase)', 'GDP (main contributors to growth)', 'GRP',
                 'GRP (compared to preceding year)']:
    nindicator = indicator.replace('GDP', 'Gross Domestic Product').replace('GRP', 'Gross Regional Product')
    for item in ['Industry breakdown', 'Sector']:
            socioecon.loc[(socioecon['file_name']=='GDP_国内生产总值')&(socioecon['indicator']==indicator)& (socioecon['category_1']==item), 'indicator'] = f'{nindicator} ({item})'
            socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})', 'category_1'] = socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})']['category_2']
            socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})', 'category_2'] = socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})']['category_3']
            socioecon.loc[socioecon['indicator']==f'{nindicator} ({item})', 'category_3'] = None
socioecon.drop(socioecon.loc[(socioecon['file_name']=='GDP_国内生产总值')&(socioecon['indicator']==indicator)].index, inplace=True)

In [52]:
for item in ['By Gender', 'By Residence']:
    socioecon.loc[(socioecon['file_name']=='Population_人口')&(socioecon['indicator']=='Population')& (socioecon['category_1']==item), 'indicator'] = f'Population ({item})'
    socioecon.loc[socioecon['indicator']==f'Population ({item})', 'category_1'] = socioecon.loc[socioecon['indicator']==f'Population ({item})']['category_2']
    socioecon.loc[socioecon['indicator']==f'Population ({item})', 'category_2'] = socioecon.loc[socioecon['indicator']==f'Population ({item})']['category_3']
    socioecon.loc[socioecon['indicator']==f'Population ({item})', 'category_3'] = None

socioecon.loc[socioecon['indicator']=='Dependency Rate', 'category_1'] = socioecon.loc[socioecon['indicator']=='Dependency Rate']['category_2']
socioecon.loc[socioecon['indicator']=='Dependency Rate', 'category_2'] = socioecon.loc[socioecon['indicator']=='Dependency Rate']['category_3']
socioecon.loc[socioecon['indicator']=='Dependency Rate', 'category_3'] = None

In [53]:
ie_inds = summary_socio[summary_socio['file_name']==summary_socio['file_name'].unique()[9]]['indicator'].unique()
ie_inds

array(['Length of Railways in Operation', 'Length of Highways',
       'Length of Navigable Inland Waterways',
       'Length of Regular Civil Aviation Routes',
       'Length of Petroleum and Gas Pipelines', 'Passenger Traffic',
       'Passenger Traffic per kilometer', 'Freight Traffic',
       'Travel Distance per passenger',
       'Average Freight Transport Distance',
       'Highspeed Railway in Operation', 'Highspeed Passenger Traffic',
       'Civil Vehicles', 'Private Vehicles', 'Air Passenger Traffic',
       'Air Freight Traffic', 'Total Air Traffic',
       'Flying Time of General Aviation'], dtype=object)

In [54]:
socioecon.loc[(socioecon['indicator']=='Civil Vehicles')&((socioecon['category_1']=='Motor Drivers')), 'indicator'] = 'Motor Drivers'
socioecon.loc[socioecon['indicator']=='Motor Drivers', 'category_1'] = socioecon.loc[socioecon['indicator']=='Motor Drivers']['category_2']
socioecon.loc[socioecon['indicator']=='Motor Drivers', 'category_2'] = None
socioecon.loc[socioecon['indicator']=='Motor Drivers', 'category_3'] = None

### Fix namings and regions

In [55]:
cat_dict = {'Number of Irrigated Areas over 10 000 Mu (set)': 'Number of Irrigated Areas over 10 000 Mu',
           'Effective Irrigated Area (10 000 hectares)': '10 000 hectares',
           'Number of Reservoirs (unit)': 'Total',
           'Capacity of Reservoirs (100 million cu.m)': 'Total',
           'Final Consumption Rate (%)': 'Final Consumption Rate',
           'Capital Formation Rate (%)': 'Capital Formation Rate',
           'Commodities and Transactions not classified elsewhere   in the SITC': 'Commodities and Transactions not classified elsewhere in the SITC',
           'Aquatic and Seawater Products(10 000 tons)': 'Aquatic and Seawater Products',
           'Synthetic Fibers Suitable for Spinning(10 000 tons)': 'Synthetic Fibers Suitable for Spinning',
           'Manganese Ores and Concentrates(10 000 tons)': 'Manganese Ores and Concentrates',
           'Chromium Ores and Concentrates(10 000 tons)': 'Chromium Ores and Concentrates',
           'Xerox and Hectograph Printing Equipment(10 000 sets)': 'Xerox and Hectograph Printing Equipment'}
socioecon.replace({'category_1': cat_dict}, inplace=True)

In [56]:
socioecon.drop(socioecon.loc[(socioecon['Region']=='a) Since 1980, the difference between the Gross Domestic Product and the Gross National Income (formerly, the Gross National Product) is the ')].index, inplace=True)
socioecon.drop(socioecon.loc[(socioecon['Region']=='    net income of primary distribution from the rest of the world.')].index, inplace=True)

In [57]:
#fix spaces in region names
reg_dict = {}
for item in list(socioecon['Region'].unique()):
    try:
        new_label = re.sub("\s\s+", " ", item)
        if new_label[0] == ' ':
            new_label = new_label[1:]
        if new_label[-1] == ' ':
            new_label = new_label[:-1]
        if item != new_label:
            reg_dict[item] = new_label
    except:
        pass

socioecon.replace({'Region':reg_dict}, inplace=True)

In [58]:
#drop rows where value = 'nan'
print(len(socioecon))
socioecon.dropna(subset=['value'], inplace=True)
print(len(socioecon))

16403
16123


### rename columns and include string translations

In [59]:
ren_dict = {'file_name': 'subgroup_en', 'Year': 'year'}
for item in socioecon.keys()[2:]:
    if item not in ['Year', 'region_type', 'value']:
        ren_dict[item] = f'{item.lower()}_en'
socioecon.rename(columns=ren_dict, inplace=True)
socioecon.drop(columns=['table_name'], inplace=True)

In [ ]:
socioecon['indicator_en'].replace({'GDP': 'Gross Domestic Product',
                                   'GNI': 'Gross National Income'}, regex=True, inplace=True)

In [60]:
socioecon['subgroup_cn'] = [x.split('_')[1] for x in socioecon['subgroup_en']]
socioecon['subgroup_en'] = [x.split('_')[0] for x in socioecon['subgroup_en']]
socioecon['group_en'] = 'Socioeconomic'
socioecon['group_cn'] = '社会经济'

In [103]:
from google_trans_new import google_translator
translator = google_translator()  

In [117]:
inds = ''
for x in socioecon['indicator_en'].unique():
    inds += f'. {x}'
translated = translator.translate(inds,lang_tgt='zh-cn') 

In [118]:
ind_dict = {}
for en, cn in zip(inds.split('. ')[1:], translated.split('。')[1:]):
    ind_dict[en] = cn

In [121]:
socioecon['indicator_cn'] = socioecon['indicator_en']
socioecon['indicator_cn'].replace(ind_dict, inplace=True)

In [138]:
for field in range(1,4):
    labels = ''
    for x in socioecon[f'category_{field}_en'].unique():
        labels += f'. {x}'
    translated = translator.translate(labels,lang_tgt='zh-cn')
    lab_dict = {}
    for en, cn in zip(labels.split('. ')[1:], translated.split('。')[1:]):
        lab_dict[en] = cn
    socioecon[f'category_{field}_cn'] = socioecon[f'category_{field}_en']
    socioecon[f'category_{field}_cn'].replace(lab_dict, inplace=True)

In [146]:
field = 1
socioecon[f'category_{field}_cn'].unique()

array([nan, 'Large and Medium-sized Tractors Number',
       'Towing Farm Machinery', 'Small Tractors Number', 'Total',
       'Nitrogenous Fertilizer', 'Phosphate Fertilizer',
       'Potash Fertilizer', 'Compound Fertilizer',
       'Number of Irrigated Areas over 10 000 Mu  (set)',
       '33 000  Hectares and Over', '20 000-33 000  Hectares',
       'Effective Irrigated Area  (10 000 hectares)',
       '33 000 Hectares and Over', '20 000-33 000 Hectares',
       'Large Reservoir', 'Medium-sized Reservoir', 'Small Reservoir',
       'State-Owned', 'Collective-Owned', 'Primary Industry',
       'Secondary Industry', 'Tertiary Industry', 'Urban', 'Rural',
       'Agriculture, Forestry, Animal Husbandry and Fishery', 'Mining',
       'Manufacturing',
       'Production and Supply of Electricity, Heat,Gas, and Water',
       'Construction', 'Wholesale and Retail Trades',
       'Transport, Storage and Post', 'Hotels and Catering Services',
       'Information Transmission, Software and 

In [142]:
for field in ['units', 'region']:
    labels = ''
    for x in socioecon[f'{field}_en'].unique():
        labels += f'. {x}'
    translated = translator.translate(labels,lang_tgt='zh-cn')
    lab_dict = {}
    for en, cn in zip(labels.split('. ')[1:], translated.split('。')[1:]):
        lab_dict[en] = cn
    socioecon[f'{field}_cn'] = socioecon[f'{field}_en']
    socioecon[f'{field}_cn'].replace(lab_dict, inplace=True)

In [132]:
socioecon = socioecon[['group_en', 'group_cn', 'subgroup_en', 'subgroup_cn',
                      'indicator_en', 'indicator_cn',
                       'category_1_en', 'category_1_cn',
                       'category_2_en', 'category_2_cn',
                       'category_3_en','category_3_cn',
                       'region_en', 'region_cn', 'region_type',
                      'year', 'value', 'units_en', 'units_cn']]

In [133]:
socioecon.groupby(['subgroup_en', 'subgroup_cn',
                      'indicator_en', 'indicator_cn',
                       'category_1_en', 'category_1_cn',
                       'category_2_en', 'category_2_cn',
                       'category_3_en','category_3_cn'], sort=False, as_index=False, dropna=False).size()

,subgroup_en,subgroup_cn,indicator_en,indicator_cn,category_1_en,category_1_cn,category_2_en,category_2_cn,category_3_en,category_3_cn,size
0,Agriculture,农业,Total Power of Agricultural Machinery,农业机械总能力,NaN,NaN,NaN,NaN,NaN,NaN,52
1,Agriculture,农业,Farm Machinery,农业机械,Large and Medium-sized Tractors Number,Large and Medium-sized Tractors Number,NaN,NaN,NaN,NaN,52
2,Agriculture,农业,Farm Machinery,农业机械,Towing Farm Machinery,Towing Farm Machinery,NaN,NaN,NaN,NaN,52
3,Agriculture,农业,Farm Machinery,农业机械,Small Tractors Number,Small Tractors Number,NaN,NaN,NaN,NaN,52
4,Agriculture,农业,Irrigated Area of Cultivated Land,灌溉面积耕地,NaN,NaN,NaN,NaN,NaN,NaN,52
...,...,...,...,...,...,...,...,...,...,...,...
818,Transportation,交通,Flying Time of General Aviation,一般航空飞行时间,Total,Total,NaN,NaN,NaN,NaN,6
819,Transportation,交通,Flying Time of General Aviation,一般航空飞行时间,Flight for Agriculture and Forestry Operation,Flight for Agriculture and Forestry Operation,Total,Total,NaN,NaN,4
820,Transportation,交通,Flying Time of General Aviation,一般航空飞行时间,Flight for Agriculture and Forestry Operation,Flight for Agriculture and Forestry Operation,Forest Protection Service,Forest Protection Service,NaN,NaN,4
821,Transportation,交通,Flying Time of General Aviation,一般航空飞行时间,Flight for Agriculture and Forestry Operation,Flight for Agriculture and Forestry Operation,Afforestation,Afforestation,NaN,NaN,4


### fixing totals

In [189]:
total_inds = []
for indicator in socioecon['indicator'].unique():
    if len(socioecon[socioecon['indicator']==indicator]['category_1'].unique()) > 1 and 'Total' in socioecon[socioecon['indicator']==indicator]['category_1'].unique():
        total_inds.append(indicator)

In [190]:
df = socioecon[socioecon['indicator']==total_inds[0]]
a = df.groupby('category_1').sum()

In [193]:
total_inds

['Consumption of Chemical Fertilizers',
 'Number of Reservoirs',
 'Capacity of Reservoirs',
 'Building Floorspace',
 'Employment',
 'Employed People in urban areas',
 'International Trade of Goods',
 'Exports by SITC',
 'Imports by SITC',
 'International Trade (by region)',
 'Length of Railways in Operation',
 'Length of Highways',
 'Length of Regular Civil Aviation Routes',
 'Passenger Traffic',
 'Passenger Traffic per kilometer',
 'Freight Traffic',
 'Travel Distance per passenger',
 'Average Freight Transport Distance',
 'Civil Vehicles',
 'Motor Drivers',
 'Private Vehicles',
 'Air Passenger Traffic',
 'Air Freight Traffic',
 'Total Air Traffic',
 'Flying Time of General Aviation']

In [199]:
socioecon[socioecon['indicator']=='Employment']['category_1'].unique()

array(['Total', 'Primary Industry', 'Secondary Industry',
       'Tertiary Industry', 'Urban', 'Rural'], dtype=object)

In [264]:
socioecon[socioecon['indicator']=='Employment']

,file_name,table_name,indicator,category_1,category_2,category_3,Region,region_type,Year,value,units
934,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1952.0,20729.000000,10 000 persons
935,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1957.0,23771.000000,10 000 persons
936,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1962.0,25910.000000,10 000 persons
937,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1965.0,28670.000000,10 000 persons
938,Employment_就业,4-2 Number of Employed Persons,Employment,Total,NaN,NaN,China,Country,1970.0,34432.000000,10 000 persons
...,...,...,...,...,...,...,...,...,...,...,...
1821,Employment_就业,1-14 分城乡就业人员年末人数,Employment,Rural,NaN,NaN,China,Country,2014.0,49.100000,Percentage
1822,Employment_就业,1-14 分城乡就业人员年末人数,Employment,Rural,NaN,NaN,China,Country,2015.0,47.800000,Percentage
1823,Employment_就业,1-14 分城乡就业人员年末人数,Employment,Rural,NaN,NaN,China,Country,2016.0,46.615466,Percentage
1824,Employment_就业,1-14 分城乡就业人员年末人数,Employment,Rural,NaN,NaN,China,Country,2017.0,45.300000,Percentage


In [276]:
tot_df = pd.DataFrame(columns=['indicator', 'units', 'region', 'year', 'total_calc', 'total_rep'])
for ind in total_inds:
    try:
        df = socioecon[socioecon['indicator']==ind]
        for unit in df['units'].unique():
            dfu = df[df['units']==unit]
            for reg in dfu['Region'].unique():
                dfur = dfu[dfu['Region']==reg]
                for year in dfur['Year'].unique():
                    dfury = dfur[dfur['Year']==year]
                    dfury_g = dfury.groupby('category_1').sum()
                    tot_dict = {'indicator': ind,
                                'units': unit,
                                'region': reg,
                                'year': year,
                                'total_calc': dfury_g['value'].sum()-dfury_g.loc['Total', 'value'], #total calculated from the sum of all categories
                                'total_rep': dfury_g.loc['Total', 'value'] #reported total
                               }
                    tot_df = tot_df.append(tot_dict, ignore_index = True)
    except:
        pass

tot_df['diff'] = ((tot_df['total_rep']-tot_df['total_calc'])*100)/tot_df['total_rep'] #difference between reported and calculated totals

In [282]:
eq_tot = []
not_eq_tot = []

for ind in total_inds:
    if ind not in tot_df['indicator'].unique():
        print(ind, 'failed')
    if ind not in tot_df[(tot_df['diff']>5)|(tot_df['diff']<-5)]['indicator'].unique():
        print(ind, 'eq')
        eq_tot.append(ind)
    if ind in tot_df[(tot_df['diff']>5)|(tot_df['diff']<-5)]['indicator'].unique():
        print(ind, 'not_eq')
        not_eq_tot.append(ind)

Consumption of Chemical Fertilizers not_eq
Number of Reservoirs eq
Capacity of Reservoirs not_eq
Building Floorspace not_eq
Employment eq
Employed People in urban areas eq
International Trade of Goods not_eq
Exports by SITC not_eq
Imports by SITC not_eq
International Trade (by region) eq
Length of Railways in Operation not_eq
Length of Highways not_eq
Length of Regular Civil Aviation Routes not_eq
Passenger Traffic eq
Passenger Traffic per kilometer eq
Freight Traffic not_eq
Travel Distance per passenger not_eq
Average Freight Transport Distance not_eq
Civil Vehicles not_eq
Motor Drivers not_eq
Private Vehicles not_eq
Air Passenger Traffic not_eq
Air Freight Traffic not_eq
Total Air Traffic eq
Flying Time of General Aviation not_eq


In [1]:
for ind in not_eq_tot:
    print(ind, tot_df[tot_df['indicator']==ind]['diff'].max(), tot_df[tot_df['indicator']==ind]['diff'].min())

NameError: name 'not_eq_tot' is not defined

In [287]:
tot_df[tot_df['indicator']=='Air Passenger Traffic']

,indicator,units,region,year,total_calc,total_rep,diff
1214,Air Passenger Traffic,10 000 persons,China,1990.0,1.660000e+03,1.660000e+03,0.000000
1215,Air Passenger Traffic,10 000 persons,China,2000.0,7.125041e+03,6.721660e+03,-6.001207
1216,Air Passenger Traffic,10 000 persons,China,2010.0,2.744152e+04,2.676914e+04,-2.511750
1217,Air Passenger Traffic,10 000 persons,China,2017.0,5.618299e+04,5.515611e+04,-1.861763
1218,Air Passenger Traffic,10 000 persons,China,2018.0,6.230086e+04,6.117377e+04,-1.842436
1219,Air Passenger Traffic,10 000 persons,China,2019.0,6.710098e+04,6.599342e+04,-1.678288
1220,Air Passenger Traffic,10 000 persons/km,China,1990.0,2.304797e+06,2.304797e+06,0.000000
1221,Air Passenger Traffic,10 000 persons/km,China,2000.0,1.020784e+07,9.705437e+06,-5.176528
1222,Air Passenger Traffic,10 000 persons/km,China,2010.0,4.137178e+07,4.038996e+07,-2.430845
1223,Air Passenger Traffic,10 000 persons/km,China,2017.0,9.661284e+07,9.513036e+07,-1.558372


In [261]:
pwd

'/Users/oscaresbri/Documents/sci_team_data_bank/Projects/green-energy-data-platform/data/Socio-economic Data'

In [262]:
socioecon.to_csv('socioecon_cleanv2.csv', index=False)